In [1]:
import pandas as pd
import numpy as np
import operator
import os
from pathlib import Path
cwd = os.getcwd() # get path of current working directory
main_folder = str(Path(cwd).parents[0]) # main folder is the lowest level in project

from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [2]:
IPUM = pd.read_stata(main_folder + "\data\IPUM.dta", convert_categoricals=False)
#IPUM

In [3]:
IPUM = IPUM[IPUM.educ != 0]

IPUM = IPUM[IPUM.incwage != np.nan]
IPUM = IPUM[IPUM.incwage != 0]
IPUM = IPUM[IPUM.incwage != 999999]

IPUM = IPUM[IPUM.wkswork1 != np.nan]
IPUM = IPUM[IPUM.wkswork1 != 0] 
IPUM = IPUM[IPUM.wkswork1 != 999999]

IPUM = IPUM[IPUM.uhrswork != np.nan]
IPUM = IPUM[IPUM.uhrswork != '00']
IPUM = IPUM[IPUM.uhrswork != 999999]

IPUM = IPUM[IPUM.wkswork1 >= 35]
IPUM = IPUM[IPUM.uhrswork >= 30]

#IPUM

In [4]:
#keep only full time 
IPUM.loc[(IPUM['citizen'] < 3), 'citizen'] = 0
IPUM.loc[(IPUM['citizen'] > 3), 'citizen'] = 0
IPUM['sex'] = IPUM['sex'] - 1
IPUM = IPUM.rename(columns={'metarea': 'statefip'})

IPUM['ind_int'] = (IPUM['ind1990']/10).round(0)
IPUM['college'] = 0 
IPUM.loc[(IPUM['educ'] >=9), 'college'] = 1
IPUM = IPUM.drop(columns=['educ'])

IPUM['sharecollege'] = 0 
IPUM['sharenoncollege'] = 0 
IPUM.loc[(IPUM['college'] == 1), 'sharecollege'] = 1
IPUM.loc[(IPUM['college'] == 0), 'sharenoncollege'] = 1

#IPUM

In [5]:
#share of college in tradables only

IPUM['sharecollegeTR'] = 0 
IPUM['sharenoncollegeTR'] = 0 

IPUM.loc[(IPUM['college'] ==1) & ((IPUM['ind_int'] >=10) & (IPUM['ind_int'] <40)), 'sharecollegeTR'] = 1
IPUM.loc[(IPUM['college'] ==0) & ((IPUM['ind_int'] >=10) & (IPUM['ind_int'] <40)), 'sharenoncollegeTR'] = 1

#IPUM

In [6]:
#*DIVIDE  tradables IN 2
IPUM['sharecollegeTR0'] = 0 
IPUM['sharenoncollegeTR0'] = 0 
IPUM.loc[(IPUM['college'] ==1) & ((IPUM['ind_int'] >=10) & (IPUM['ind_int'] <= 20)), 'sharecollegeTR0'] = 1
IPUM.loc[(IPUM['college'] ==0) & ((IPUM['ind_int'] >=10) & (IPUM['ind_int'] <= 20)), 'sharenoncollegeTR0'] = 1

IPUM['sharecollegeTR1'] = 0 
IPUM['sharenoncollegeTR1'] = 0 
IPUM.loc[(IPUM['college'] ==1) & ((IPUM['ind_int'] >=21) & (IPUM['ind_int'] <40)), 'sharecollegeTR1'] = 1
IPUM.loc[(IPUM['college'] ==0) & ((IPUM['ind_int'] >=21) & (IPUM['ind_int'] <40)), 'sharenoncollegeTR1'] = 1
#IPUM

In [7]:
#******************************IVS
#1) NATIONAL GROWTH IN INCOME AND EDUCATION BY INDUSTRY


In [8]:
IPUM1 = IPUM.rename(columns={'sharecollege':'IV', 'sharenoncollege':'IV1', 'sharecollegeTR':'IVTR',
                            'sharenoncollegeTR':'IV1TR', 'sharecollegeTR0':'IVTR0','sharenoncollegeTR0':'IV1TR0',
                            'college':'tot'})

In [9]:
IPUM2 = IPUM1.groupby(['ind_int','year'])['IV','IV1','IVTR','IV1TR','IVTR0','IV1TR0'].sum().sort_values(by=['ind_int','year'])
IPUM2['tot'] =IPUM2.sum(axis = 1, skipna = True) 
#IPUM2

<ipython-input-9-21b6abf14842>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  IPUM2 = IPUM1.groupby(['ind_int','year'])['IV','IV1','IVTR','IV1TR','IVTR0','IV1TR0'].sum().sort_values(by=['ind_int','year'])


In [10]:
IPUM2['totedu_IVHLTR0'] = np.log(IPUM2['IVTR0']/IPUM2['IV1TR0'])
IPUM2['totedu_IVHLTR'] = np.log(IPUM2['IVTR']/IPUM2['IV1TR'])
IPUM2['totedu_IVHL'] = np.log(IPUM2['IV']/IPUM2['IV1'])
#IPUM2

In [11]:
#Summary statistics
IPUM2[['IV', 'IV1', 'IVTR', 'IV1TR', 'IVTR0', 'IV1TR0', 'tot',
       'totedu_IVHLTR0', 'totedu_IVHLTR', 'totedu_IVHL']].describe()

,IV,IV1,IVTR,IV1TR,IVTR0,IV1TR0,tot,totedu_IVHLTR0,totedu_IVHLTR,totedu_IVHL
count,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,33.000000,89.000000,267.000000
mean,9259.202247,22362.947566,1344.513109,4976.172285,462.752809,1543.299625,39948.887640,-1.417830,-1.515124,-1.259779
std,18008.209981,24622.321514,3084.251737,9663.154627,1840.661353,5062.914770,41076.567307,0.599941,0.548196,0.700431
min,167.000000,744.000000,0.000000,0.000000,0.000000,0.000000,924.000000,-2.315354,-2.413098,-2.874848
25%,1799.000000,8798.000000,0.000000,0.000000,0.000000,0.000000,14308.500000,-1.927065,-1.946003,-1.830653
50%,3950.000000,14886.000000,0.000000,0.000000,0.000000,0.000000,28378.000000,-1.560484,-1.653133,-1.294357
75%,9286.000000,26153.000000,1308.000000,7921.000000,0.000000,0.000000,46350.000000,-0.855142,-1.043983,-0.772361
max,182648.000000,188515.000000,20394.000000,58191.000000,16594.000000,37827.000000,291741.000000,0.051551,0.051551,0.744059


In [12]:
IPUM3 =IPUM2[['totedu_IVHLTR0', 'totedu_IVHLTR', 'totedu_IVHL']]
IPUM3

totedu_IVHLTR0  totedu_IVHLTR  totedu_IVHL
ind_int year                                            
0.0     1980             NaN            NaN    -0.910415
1.0     1980             NaN            NaN    -1.908506
        1990             NaN            NaN    -1.642415
        2000             NaN            NaN    -1.483897
2.0     1980             NaN            NaN    -1.560711
...                      ...            ...          ...
94.0    2000             NaN            NaN    -0.803735
95.0    1990             NaN            NaN    -1.306777
        2000             NaN            NaN    -1.221416
96.0    1990             NaN            NaN    -1.511369
        2000             NaN            NaN    -0.966594

[267 rows x 3 columns]

In [13]:
#**  GROWTH IN EDUCATION BY INDUSTRY-YEAR
#LAG
IPUM3['totedu_IVHL_lag'] = IPUM3.groupby(['ind_int'])['totedu_IVHL'].shift(1)
IPUM3['totedu_IVHLTR_lag'] = IPUM3.groupby(['ind_int'])['totedu_IVHLTR'].shift(1)
IPUM3['totedu_IVHLTR0_lag'] = IPUM3.groupby(['ind_int'])['totedu_IVHLTR0'].shift(1)

IPUM3['dtotedu_IVHL'] = IPUM3['totedu_IVHL'] - IPUM3['totedu_IVHL_lag']
IPUM3['dtotedu_IVHLTR'] = IPUM3['totedu_IVHLTR']-IPUM3['totedu_IVHLTR_lag']
IPUM3['dtotedu_IVHLTR0'] = IPUM3['totedu_IVHLTR0']-IPUM3['totedu_IVHLTR0_lag']

IPUM4 = IPUM3[['dtotedu_IVHL','dtotedu_IVHLTR','dtotedu_IVHLTR0']]
IPUM4 

<ipython-input-13-dce1998d775c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IPUM3['totedu_IVHL_lag'] = IPUM3.groupby(['ind_int'])['totedu_IVHL'].shift(1)
<ipython-input-13-dce1998d775c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IPUM3['totedu_IVHLTR_lag'] = IPUM3.groupby(['ind_int'])['totedu_IVHLTR'].shift(1)
<ipython-input-13-dce1998d775c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

dtotedu_IVHL  dtotedu_IVHLTR  dtotedu_IVHLTR0
ind_int year                                               
0.0     1980           NaN             NaN              NaN
1.0     1980           NaN             NaN              NaN
        1990      0.266091             NaN              NaN
        2000      0.158517             NaN              NaN
2.0     1980           NaN             NaN              NaN
...                    ...             ...              ...
94.0    2000      0.137000             NaN              NaN
95.0    1990           NaN             NaN              NaN
        2000      0.085361             NaN              NaN
96.0    1990           NaN             NaN              NaN
        2000      0.544775             NaN              NaN

[267 rows x 3 columns]

In [14]:
IPUM4.to_stata('tmmp4')

In [15]:
#**IV for tradables only 
IPUM

,year,statefip,perwt,age,sex,race,citizen,ind1990,wkswork1,uhrswork,...,ind_int,college,sharecollege,sharenoncollege,sharecollegeTR,sharenoncollegeTR,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1
0,1980,38,20,50,0,1,0,42,52,38,...,4.0,1,1,0,0,0,0,0,0,0
1,1980,38,20,23,0,1,0,420,52,40,...,42.0,0,0,1,0,0,0,0,0,0
2,1980,38,20,58,1,1,0,412,48,40,...,41.0,0,0,1,0,0,0,0,0,0
3,1980,38,20,59,0,1,0,421,50,50,...,42.0,0,0,1,0,0,0,0,0,0
7,1980,38,20,47,1,1,0,512,48,40,...,51.0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11212454,2000,856,25,53,1,1,0,591,52,40,...,59.0,0,0,1,0,0,0,0,0,0
11212455,2000,856,38,44,1,1,0,601,52,60,...,60.0,0,0,1,0,0,0,0,0,0
11212456,2000,856,21,51,0,1,0,601,52,65,...,60.0,0,0,1,0,0,0,0,0,0
11212457,2000,856,25,45,1,3,0,111,52,40,...,11.0,0,0,1,0,1,0,1,0,0


In [16]:
IPUM5 = IPUM.loc[operator.and_(IPUM['ind_int'] >=10, IPUM['ind_int'] <40)] 
IPUM5

,year,statefip,perwt,age,sex,race,citizen,ind1990,wkswork1,uhrswork,...,ind_int,college,sharecollege,sharenoncollege,sharecollegeTR,sharenoncollegeTR,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1
25,1980,38,20,43,0,1,0,200,46,55,...,20.0,0,0,1,0,1,0,1,0,0
55,1980,38,20,34,1,1,0,320,45,40,...,32.0,0,0,1,0,1,0,0,0,1
170,1980,38,20,23,1,1,3,271,52,40,...,27.0,0,0,1,0,1,0,0,0,1
218,1980,38,20,36,0,1,0,172,52,39,...,17.0,0,0,1,0,1,0,1,0,0
287,1980,38,20,30,0,2,0,200,52,40,...,20.0,0,0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11212418,2000,856,25,44,1,7,0,351,52,50,...,35.0,0,0,1,0,1,0,0,0,1
11212419,2000,856,33,38,1,1,0,312,52,40,...,31.0,0,0,1,0,1,0,0,0,1
11212436,2000,856,29,46,0,1,0,172,52,40,...,17.0,0,0,1,0,1,0,1,0,0
11212457,2000,856,25,45,1,3,0,111,52,40,...,11.0,0,0,1,0,1,0,1,0,0


In [17]:
IPUM5['empl0'] = 1
IPUM5['empl'] = IPUM5.groupby(['statefip', 'ind_int', 'year'])['empl0'].transform('sum')
IPUM5 = IPUM5.sort_values(by=['statefip', 'ind_int', 'year']).drop(columns=['empl0'])

IPUM5['totempl0'] = 1
IPUM5['totempl'] = IPUM5.groupby(['statefip', 'year'])['totempl0'].transform('sum')
IPUM5 = IPUM5.sort_values(by=['statefip', 'year']).drop(columns=['totempl0'])

IPUM5

<ipython-input-17-96e85eec0399>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IPUM5['empl0'] = 1
<ipython-input-17-96e85eec0399>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IPUM5['empl'] = IPUM5.groupby(['statefip', 'ind_int', 'year'])['empl0'].transform('sum')


,year,statefip,perwt,age,sex,race,citizen,ind1990,wkswork1,uhrswork,...,sharecollege,sharenoncollege,sharecollegeTR,sharenoncollegeTR,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1,empl,totempl
2882268,1980,4,20,45,0,1,0,100,52,40,...,0,1,0,1,0,1,0,0,9,215
2882302,1980,4,20,32,0,1,0,100,52,50,...,0,1,0,1,0,1,0,0,9,215
2882543,1980,4,20,24,0,2,0,100,52,40,...,0,1,0,1,0,1,0,0,9,215
2882801,1980,4,20,22,0,1,0,100,52,40,...,0,1,0,1,0,1,0,0,9,215
2883155,1980,4,20,19,0,1,0,100,52,40,...,0,1,0,1,0,1,0,0,9,215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8492059,2000,936,22,41,0,1,0,392,52,45,...,0,1,0,1,0,0,0,1,8,107
8492451,2000,936,19,32,0,6,0,392,40,40,...,0,1,0,1,0,0,0,1,8,107
8492848,2000,936,37,47,0,1,0,392,51,40,...,0,1,0,1,0,0,0,1,8,107
8493315,2000,936,11,21,0,1,3,392,52,40,...,0,1,0,1,0,0,0,1,8,107


In [18]:
IPUM5.loc[IPUM5['year'] == 1980, 'shareTR'] = IPUM5['empl']/IPUM5['totempl']
IPUM5

,year,statefip,perwt,age,sex,race,citizen,ind1990,wkswork1,uhrswork,...,sharenoncollege,sharecollegeTR,sharenoncollegeTR,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1,empl,totempl,shareTR
2882268,1980,4,20,45,0,1,0,100,52,40,...,1,0,1,0,1,0,0,9,215,0.04186
2882302,1980,4,20,32,0,1,0,100,52,50,...,1,0,1,0,1,0,0,9,215,0.04186
2882543,1980,4,20,24,0,2,0,100,52,40,...,1,0,1,0,1,0,0,9,215,0.04186
2882801,1980,4,20,22,0,1,0,100,52,40,...,1,0,1,0,1,0,0,9,215,0.04186
2883155,1980,4,20,19,0,1,0,100,52,40,...,1,0,1,0,1,0,0,9,215,0.04186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8492059,2000,936,22,41,0,1,0,392,52,45,...,1,0,1,0,0,0,1,8,107,NaN
8492451,2000,936,19,32,0,6,0,392,40,40,...,1,0,1,0,0,0,1,8,107,NaN
8492848,2000,936,37,47,0,1,0,392,51,40,...,1,0,1,0,0,0,1,8,107,NaN
8493315,2000,936,11,21,0,1,3,392,52,40,...,1,0,1,0,0,0,1,8,107,NaN


In [19]:
IPUM5['share1'] = IPUM5.groupby(['statefip', 'ind_int'])['shareTR'].transform('max')
IPUM5 = IPUM5.drop(columns=['shareTR'])
IPUM5 = IPUM5.rename(columns={'share1': 'shareTR'})
IPUM5

,year,statefip,perwt,age,sex,race,citizen,ind1990,wkswork1,uhrswork,...,sharenoncollege,sharecollegeTR,sharenoncollegeTR,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1,empl,totempl,shareTR
2882268,1980,4,20,45,0,1,0,100,52,40,...,1,0,1,0,1,0,0,9,215,0.04186
2882302,1980,4,20,32,0,1,0,100,52,50,...,1,0,1,0,1,0,0,9,215,0.04186
2882543,1980,4,20,24,0,2,0,100,52,40,...,1,0,1,0,1,0,0,9,215,0.04186
2882801,1980,4,20,22,0,1,0,100,52,40,...,1,0,1,0,1,0,0,9,215,0.04186
2883155,1980,4,20,19,0,1,0,100,52,40,...,1,0,1,0,1,0,0,9,215,0.04186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8492059,2000,936,22,41,0,1,0,392,52,45,...,1,0,1,0,0,0,1,8,107,NaN
8492451,2000,936,19,32,0,6,0,392,40,40,...,1,0,1,0,0,0,1,8,107,NaN
8492848,2000,936,37,47,0,1,0,392,51,40,...,1,0,1,0,0,0,1,8,107,NaN
8493315,2000,936,11,21,0,1,3,392,52,40,...,1,0,1,0,0,0,1,8,107,NaN


In [20]:
tmmp4 = pd.read_stata('tmmp4')
tmmp4 

,ind_int,year,dtotedu_IVHL,dtotedu_IVHLTR,dtotedu_IVHLTR0
0,0.0,1980,NaN,NaN,NaN
1,1.0,1980,NaN,NaN,NaN
2,1.0,1990,0.266091,NaN,NaN
3,1.0,2000,0.158517,NaN,NaN
4,2.0,1980,NaN,NaN,NaN
...,...,...,...,...,...
262,94.0,2000,0.137000,NaN,NaN
263,95.0,1990,NaN,NaN,NaN
264,95.0,2000,0.085361,NaN,NaN
265,96.0,1990,NaN,NaN,NaN


In [21]:
IPUM6 = pd.merge(IPUM5, tmmp4, on=['ind_int','year'])
IPUM6

,year,statefip,perwt,age,sex,race,citizen,ind1990,wkswork1,uhrswork,...,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1,empl,totempl,shareTR,dtotedu_IVHL,dtotedu_IVHLTR,dtotedu_IVHLTR0
0,1980,4,20,45,0,1,0,100,52,40,...,0,1,0,0,9,215,0.041860,NaN,NaN,NaN
1,1980,4,20,32,0,1,0,100,52,50,...,0,1,0,0,9,215,0.041860,NaN,NaN,NaN
2,1980,4,20,24,0,2,0,100,52,40,...,0,1,0,0,9,215,0.041860,NaN,NaN,NaN
3,1980,4,20,22,0,1,0,100,52,40,...,0,1,0,0,9,215,0.041860,NaN,NaN,NaN
4,1980,4,20,19,0,1,0,100,52,40,...,0,1,0,0,9,215,0.041860,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1687618,2000,932,34,29,1,1,0,291,52,40,...,0,0,0,1,46,2760,0.026899,-0.137525,-0.137525,NaN
1687619,2000,932,22,36,0,1,0,291,52,50,...,0,0,0,1,46,2760,0.026899,-0.137525,-0.137525,NaN
1687620,2000,932,6,43,0,1,0,291,52,45,...,0,0,0,1,46,2760,0.026899,-0.137525,-0.137525,NaN
1687621,2000,932,29,23,0,1,0,291,52,45,...,0,0,0,1,46,2760,0.026899,-0.137525,-0.137525,NaN


In [22]:
IPUM6 = IPUM6.groupby(['statefip', 'ind_int', 'year'])['shareTR', 'dtotedu_IVHLTR'].mean().fillna(0)
IPUM6 

<ipython-input-22-37545ab594a2>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  IPUM6 = IPUM6.groupby(['statefip', 'ind_int', 'year'])['shareTR', 'dtotedu_IVHLTR'].mean().fillna(0)


shareTR  dtotedu_IVHLTR
statefip ind_int year                          
4        10.0    1980  0.041860        0.000000
                 1990  0.041860        0.025144
                 2000  0.041860       -0.045922
         11.0    1980  0.060465        0.000000
                 1990  0.060465        0.050062
...                         ...             ...
936      37.0    1990  0.000000        0.199771
                 2000  0.000000        0.331961
         38.0    1990  0.000000        0.099198
         39.0    1990  0.000000        0.153049
                 2000  0.000000        0.089937

[21402 rows x 2 columns]

In [23]:
IPUM6['ddtotedu_IVHLTR'] = IPUM6['dtotedu_IVHLTR']*IPUM6['shareTR']
IPUM6

shareTR  dtotedu_IVHLTR  ddtotedu_IVHLTR
statefip ind_int year                                           
4        10.0    1980  0.041860        0.000000         0.000000
                 1990  0.041860        0.025144         0.001053
                 2000  0.041860       -0.045922        -0.001922
         11.0    1980  0.060465        0.000000         0.000000
                 1990  0.060465        0.050062         0.003027
...                         ...             ...              ...
936      37.0    1990  0.000000        0.199771         0.000000
                 2000  0.000000        0.331961         0.000000
         38.0    1990  0.000000        0.099198         0.000000
         39.0    1990  0.000000        0.153049         0.000000
                 2000  0.000000        0.089937         0.000000

[21402 rows x 3 columns]

In [24]:
IPUM7 = IPUM6.groupby(['statefip','year'])['ddtotedu_IVHLTR'].agg('sum')
IPUM7 = pd.DataFrame(IPUM7)
IPUM7 

ddtotedu_IVHLTR
statefip year                 
4        1980         0.000000
         1990         0.082758
         2000         0.180849
8        1980         0.000000
         1990         0.047387
...                        ...
932      2000         0.131639
934      1990         0.000000
         2000         0.000000
936      1990         0.000000
         2000         0.000000

[787 rows x 1 columns]

In [25]:
IPUM7.to_stata('IV1')

In [26]:
#**IV2 for subgroup of tradables

In [27]:
IPUM8 = IPUM.loc[operator.and_(IPUM['ind_int'] >=10, IPUM['ind_int'] <= 20)] 
IPUM8

,year,statefip,perwt,age,sex,race,citizen,ind1990,wkswork1,uhrswork,...,ind_int,college,sharecollege,sharenoncollege,sharecollegeTR,sharenoncollegeTR,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1
25,1980,38,20,43,0,1,0,200,46,55,...,20.0,0,0,1,0,1,0,1,0,0
218,1980,38,20,36,0,1,0,172,52,39,...,17.0,0,0,1,0,1,0,1,0,0
287,1980,38,20,30,0,2,0,200,52,40,...,20.0,0,0,1,0,1,0,1,0,0
330,1980,38,20,47,0,1,0,200,43,84,...,20.0,0,0,1,0,1,0,1,0,0
336,1980,38,20,40,0,1,0,172,51,39,...,17.0,0,0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11212327,2000,856,29,36,1,1,0,200,52,50,...,20.0,1,1,0,1,0,1,0,0,0
11212360,2000,856,29,42,0,1,0,172,52,40,...,17.0,0,0,1,0,1,0,1,0,0
11212417,2000,856,13,49,0,1,0,120,52,60,...,12.0,0,0,1,0,1,0,1,0,0
11212436,2000,856,29,46,0,1,0,172,52,40,...,17.0,0,0,1,0,1,0,1,0,0


In [28]:
IPUM8['empl0'] = 1
IPUM8['empl'] = IPUM8.groupby(['statefip', 'ind_int', 'year'])['empl0'].transform('sum')
IPUM8 = IPUM8.sort_values(by=['statefip', 'ind_int', 'year']).drop(columns=['empl0'])

IPUM8['totempl0'] = 1
IPUM8['totempl'] = IPUM8.groupby(['statefip', 'year'])['totempl0'].transform('sum')
IPUM8 = IPUM8.sort_values(by=['statefip', 'year']).drop(columns=['totempl0'])

IPUM8

<ipython-input-28-268c5e98f570>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IPUM8['empl0'] = 1
<ipython-input-28-268c5e98f570>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IPUM8['empl'] = IPUM8.groupby(['statefip', 'ind_int', 'year'])['empl0'].transform('sum')


,year,statefip,perwt,age,sex,race,citizen,ind1990,wkswork1,uhrswork,...,sharecollege,sharenoncollege,sharecollegeTR,sharenoncollegeTR,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1,empl,totempl
2882268,1980,4,20,45,0,1,0,100,52,40,...,0,1,0,1,0,1,0,0,9,72
2882302,1980,4,20,32,0,1,0,100,52,50,...,0,1,0,1,0,1,0,0,9,72
2882543,1980,4,20,24,0,2,0,100,52,40,...,0,1,0,1,0,1,0,0,9,72
2882801,1980,4,20,22,0,1,0,100,52,40,...,0,1,0,1,0,1,0,0,9,72
2883155,1980,4,20,19,0,1,0,100,52,40,...,0,1,0,1,0,1,0,0,9,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8491482,2000,936,8,25,0,7,0,192,52,50,...,1,0,1,0,1,0,0,0,7,43
8492529,2000,936,16,23,0,7,0,191,52,40,...,0,1,0,1,0,1,0,0,7,43
8493081,2000,936,5,46,0,1,0,192,52,40,...,0,1,0,1,0,1,0,0,7,43
8493253,2000,936,24,36,0,1,3,191,52,40,...,1,0,1,0,1,0,0,0,7,43


In [29]:
IPUM8.loc[IPUM8['year'] == 1980, 'shareTR0'] = IPUM8['empl']/IPUM8['totempl']
IPUM8

,year,statefip,perwt,age,sex,race,citizen,ind1990,wkswork1,uhrswork,...,sharenoncollege,sharecollegeTR,sharenoncollegeTR,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1,empl,totempl,shareTR0
2882268,1980,4,20,45,0,1,0,100,52,40,...,1,0,1,0,1,0,0,9,72,0.125
2882302,1980,4,20,32,0,1,0,100,52,50,...,1,0,1,0,1,0,0,9,72,0.125
2882543,1980,4,20,24,0,2,0,100,52,40,...,1,0,1,0,1,0,0,9,72,0.125
2882801,1980,4,20,22,0,1,0,100,52,40,...,1,0,1,0,1,0,0,9,72,0.125
2883155,1980,4,20,19,0,1,0,100,52,40,...,1,0,1,0,1,0,0,9,72,0.125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8491482,2000,936,8,25,0,7,0,192,52,50,...,0,1,0,1,0,0,0,7,43,NaN
8492529,2000,936,16,23,0,7,0,191,52,40,...,1,0,1,0,1,0,0,7,43,NaN
8493081,2000,936,5,46,0,1,0,192,52,40,...,1,0,1,0,1,0,0,7,43,NaN
8493253,2000,936,24,36,0,1,3,191,52,40,...,0,1,0,1,0,0,0,7,43,NaN


In [30]:
IPUM8['share1'] = IPUM8.groupby(['statefip', 'ind_int'])['shareTR0'].transform('max')
IPUM8 = IPUM8.drop(columns=['shareTR0'])
IPUM8 = IPUM8.rename(columns={'share1': 'shareTR0'})
IPUM8

,year,statefip,perwt,age,sex,race,citizen,ind1990,wkswork1,uhrswork,...,sharenoncollege,sharecollegeTR,sharenoncollegeTR,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1,empl,totempl,shareTR0
2882268,1980,4,20,45,0,1,0,100,52,40,...,1,0,1,0,1,0,0,9,72,0.125
2882302,1980,4,20,32,0,1,0,100,52,50,...,1,0,1,0,1,0,0,9,72,0.125
2882543,1980,4,20,24,0,2,0,100,52,40,...,1,0,1,0,1,0,0,9,72,0.125
2882801,1980,4,20,22,0,1,0,100,52,40,...,1,0,1,0,1,0,0,9,72,0.125
2883155,1980,4,20,19,0,1,0,100,52,40,...,1,0,1,0,1,0,0,9,72,0.125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8491482,2000,936,8,25,0,7,0,192,52,50,...,0,1,0,1,0,0,0,7,43,NaN
8492529,2000,936,16,23,0,7,0,191,52,40,...,1,0,1,0,1,0,0,7,43,NaN
8493081,2000,936,5,46,0,1,0,192,52,40,...,1,0,1,0,1,0,0,7,43,NaN
8493253,2000,936,24,36,0,1,3,191,52,40,...,0,1,0,1,0,0,0,7,43,NaN


In [31]:
IPUM9 = pd.merge(IPUM8, tmmp4, on=['ind_int','year'])
IPUM9

,year,statefip,perwt,age,sex,race,citizen,ind1990,wkswork1,uhrswork,...,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1,empl,totempl,shareTR0,dtotedu_IVHL,dtotedu_IVHLTR,dtotedu_IVHLTR0
0,1980,4,20,45,0,1,0,100,52,40,...,0,1,0,0,9,72,0.125000,NaN,NaN,NaN
1,1980,4,20,32,0,1,0,100,52,50,...,0,1,0,0,9,72,0.125000,NaN,NaN,NaN
2,1980,4,20,24,0,2,0,100,52,40,...,0,1,0,0,9,72,0.125000,NaN,NaN,NaN
3,1980,4,20,22,0,1,0,100,52,40,...,0,1,0,0,9,72,0.125000,NaN,NaN,NaN
4,1980,4,20,19,0,1,0,100,52,40,...,0,1,0,0,9,72,0.125000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535611,2000,932,16,46,0,1,0,180,52,40,...,0,1,0,0,18,285,0.087379,0.658417,0.658417,0.658417
535612,2000,932,6,20,0,1,0,180,47,47,...,0,1,0,0,18,285,0.087379,0.658417,0.658417,0.658417
535613,2000,934,16,31,1,6,0,181,52,40,...,1,0,0,0,3,90,NaN,0.658417,0.658417,0.658417
535614,2000,934,16,29,0,7,3,181,51,40,...,0,1,0,0,3,90,NaN,0.658417,0.658417,0.658417


In [32]:
IPUM10 = IPUM9.groupby(['statefip', 'ind_int', 'year'])['shareTR0', 'dtotedu_IVHLTR0'].mean().fillna(0)
IPUM10 

<ipython-input-32-e30f3d13a04c>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  IPUM10 = IPUM9.groupby(['statefip', 'ind_int', 'year'])['shareTR0', 'dtotedu_IVHLTR0'].mean().fillna(0)


shareTR0  dtotedu_IVHLTR0
statefip ind_int year                           
4        10.0    1980  0.125000         0.000000
                 1990  0.125000         0.025144
                 2000  0.125000        -0.045922
         11.0    1980  0.180556         0.000000
                 1990  0.180556         0.050062
...                         ...              ...
936      17.0    1990  0.000000         0.095214
                 2000  0.000000         0.280926
         19.0    1990  0.000000         0.064201
                 2000  0.000000         0.149621
         20.0    1990  0.000000        -0.007911

[7812 rows x 2 columns]

In [33]:
IPUM10['ddtotedu_IVHLTR0'] = IPUM10['dtotedu_IVHLTR0']*IPUM10['shareTR0']
IPUM10

shareTR0  dtotedu_IVHLTR0  ddtotedu_IVHLTR0
statefip ind_int year                                             
4        10.0    1980  0.125000         0.000000          0.000000
                 1990  0.125000         0.025144          0.003143
                 2000  0.125000        -0.045922         -0.005740
         11.0    1980  0.180556         0.000000          0.000000
                 1990  0.180556         0.050062          0.009039
...                         ...              ...               ...
936      17.0    1990  0.000000         0.095214          0.000000
                 2000  0.000000         0.280926          0.000000
         19.0    1990  0.000000         0.064201          0.000000
                 2000  0.000000         0.149621          0.000000
         20.0    1990  0.000000        -0.007911         -0.000000

[7812 rows x 3 columns]

In [34]:
IPUM11 = IPUM10.groupby(['statefip','year'])['ddtotedu_IVHLTR0'].agg('sum')
IPUM11 = pd.DataFrame(IPUM11)
IPUM11

ddtotedu_IVHLTR0
statefip year                  
4        1980          0.000000
         1990          0.085771
         2000          0.211268
8        1980          0.000000
         1990          0.094604
...                         ...
932      2000          0.248488
934      1990          0.000000
         2000          0.000000
936      1990          0.000000
         2000          0.000000

[787 rows x 1 columns]

In [35]:
IPUM11.to_stata('IV2')

In [36]:
#merging
IV1 = pd.read_stata('IV1')
IV2 = pd.read_stata('IV2')
IPUMIV1= pd.merge(IPUM, IV1, on=['statefip','year'])
IPUMIV1IV2 = pd.merge(IPUMIV1, IV2, on=['statefip','year'])
IPUMIV1IV2 

,year,statefip,perwt,age,sex,race,citizen,ind1990,wkswork1,uhrswork,...,sharecollege,sharenoncollege,sharecollegeTR,sharenoncollegeTR,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1,ddtotedu_IVHLTR,ddtotedu_IVHLTR0
0,1980,38,20,50,0,1,0,42,52,38,...,1,0,0,0,0,0,0,0,0.000000,0.000000
1,1980,38,20,23,0,1,0,420,52,40,...,0,1,0,0,0,0,0,0,0.000000,0.000000
2,1980,38,20,58,1,1,0,412,48,40,...,0,1,0,0,0,0,0,0,0.000000,0.000000
3,1980,38,20,59,0,1,0,421,50,50,...,0,1,0,0,0,0,0,0,0.000000,0.000000
4,1980,38,20,47,1,1,0,512,48,40,...,0,1,0,0,0,0,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8443109,2000,856,25,53,1,1,0,591,52,40,...,0,1,0,0,0,0,0,0,0.201391,0.194577
8443110,2000,856,38,44,1,1,0,601,52,60,...,0,1,0,0,0,0,0,0,0.201391,0.194577
8443111,2000,856,21,51,0,1,0,601,52,65,...,0,1,0,0,0,0,0,0,0.201391,0.194577
8443112,2000,856,25,45,1,3,0,111,52,40,...,0,1,0,1,0,1,0,0,0.201391,0.194577


In [37]:
#“The kernel appears to have died. It will restart automatically.”
'''
### INDUSTRY MORE REFINED
IPUMIV1IV2['ind'] = np.nan

IPUMIV1IV2.loc[np.logical_or(IPUMIV1IV2['ind_int'] == 10,IPUMIV1IV2['ind_int'] == 11,IPUMIV1IV2['ind_int'] == 12), 'ind'] = 1 #food home
IPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 641, 'ind'] = 2 #food out
IPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 130, 'ind'] = 3 #tobacco
IPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 120, 'ind'] = 4 #alchol off premises
IPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 9999999999999, 'ind'] = 5 #niteclub
IPUMIV1IV2.loc[np.logical_or(IPUMIV1IV2['ind_int'] == 15,IPUMIV1IV2['ind_int'] == 22), 'ind'] = 6 #clothing shoes
IPUMIV1IV2.loc[np.logical_or(IPUMIV1IV2['ind1990'] == 782,IPUMIV1IV2['ind1990'] == 771), 'ind'] = 7 #clothing services
IPUMIV1IV2.loc[np.logical_or(IPUMIV1IV2['ind1990'] == 381,IPUMIV1IV2['ind1990'] == 660), 'ind'] = 8 #jewelry prof equipment
IPUMIV1IV2.loc[np.logical_and(IPUMIV1IV2['ind1990'] >= 761,IPUMIV1IV2['ind1990'] <= 781), 'ind'] = 9 #beauty shops
IPUMIV1IV2.loc[IPUMIV1IV2['ind_int'] == 6, 'ind'] = 10 #housing

IPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 501, 'ind'] = 11 #furniture household appliances
IPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 450, 'ind'] = 12 #electricity
IPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 451, 'ind'] = 13 #gas
IPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 470, 'ind'] = 14 #water
IPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 42, 'ind'] = 15 #fuel oil
IPUMIV1IV2.loc[IPUMIV1IV2['ind_int'] == 44, 'ind'] = 16 #telephone
IPUMIV1IV2.loc[np.logical_and(IPUMIV1IV2['ind1990'] >= 862,IPUMIV1IV2['ind1990'] <= 870), 'ind'] = 17 #domestic services
IPUMIV1IV2.loc[IPUMIV1IV2['ind_int'] == 18, 'ind'] = 18 #drugs
IPUMIV1IV2.loc[IPUMIV1IV2['ind_int'] == 37, 'ind']= 19 #oftamic products
IPUMIV1IV2.loc[np.logical_and(IPUMIV1IV2['ind1990'] >= 812,IPUMIV1IV2['ind1990'] <= 830), 'ind'] = 20 #doctors

IPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 831, 'ind'] = 21 #hospitals
IPUMIV1IV2.loc[IPUMIV1IV2['ind_int'] == 71, 'ind'] = 22 #insurance
IPUMIV1IV2.loc[IPUMIV1IV2['ind_int'] == 74, 'ind'] = 23 #+BUSINESS SERV.39
IPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 9999999999999, 'ind'] = 24 #insurance
IPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 315, 'ind'] = 25 #auto parts
IPUMIV1IV2.loc[IPUMIV1IV2['ind_int'] == 75, 'ind']= 26 #auto service
IPUMIV1IV2.loc[np.logical_and(IPUMIV1IV2['ind1990'] == 200,IPUMIV1IV2['ind1990'] ==201), 'ind'] = 27 #fuel
IPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 9999999999999, 'ind'] = 28 #auto insur
IPUMIV1IV2.loc[np.logical_and(IPUMIV1IV2['ind1990'] == 401,IPUMIV1IV2['ind1990'] ==400), 'ind'] = 29 #roads
IPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 402, 'ind'] = 30 #taxi bus railway

IPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 421, 'ind'] = 31 #air fares
IPUMIV1IV2.loc[np.logical_or(IPUMIV1IV2['ind1990'] == 172,IPUMIV1IV2['ind1990'] ==171), 'ind'] = 32
IPUMIV1IV2.loc[np.logical_or(IPUMIV1IV2['ind1990'] == 370,IPUMIV1IV2['ind1990'] ==380), 'ind'] = 33
IPUMIV1IV2.loc[np.logical_or(IPUMIV1IV2['ind_int'] == 80,IPUMIV1IV2['ind1990'] ==810), 'ind'] = 34 #oth recreation
IPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 850, 'ind'] = 35 #high educ
IPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 842, 'ind'] = 36 #elem educ
IPUMIV1IV2.loc[np.logical_and(IPUMIV1IV2['ind1990'] >= 851,IPUMIV1IV2['ind1990'] <=861), 'ind'] = 37 #other educ
IPUMIV1IV2.loc[np.logical_and(IPUMIV1IV2['ind1990'] >= 871,IPUMIV1IV2['ind1990'] <=881), 'ind'] = 38 #relig and welfare                           

'''

"\n### INDUSTRY MORE REFINED\nIPUMIV1IV2['ind'] = np.nan\n\nIPUMIV1IV2.loc[np.logical_or(IPUMIV1IV2['ind_int'] == 10,IPUMIV1IV2['ind_int'] == 11,IPUMIV1IV2['ind_int'] == 12), 'ind'] = 1 #food home\nIPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 641, 'ind'] = 2 #food out\nIPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 130, 'ind'] = 3 #tobacco\nIPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 120, 'ind'] = 4 #alchol off premises\nIPUMIV1IV2.loc[IPUMIV1IV2['ind1990'] == 9999999999999, 'ind'] = 5 #niteclub\nIPUMIV1IV2.loc[np.logical_or(IPUMIV1IV2['ind_int'] == 15,IPUMIV1IV2['ind_int'] == 22), 'ind'] = 6 #clothing shoes\nIPUMIV1IV2.loc[np.logical_or(IPUMIV1IV2['ind1990'] == 782,IPUMIV1IV2['ind1990'] == 771), 'ind'] = 7 #clothing services\nIPUMIV1IV2.loc[np.logical_or(IPUMIV1IV2['ind1990'] == 381,IPUMIV1IV2['ind1990'] == 660), 'ind'] = 8 #jewelry prof equipment\nIPUMIV1IV2.loc[np.logical_and(IPUMIV1IV2['ind1990'] >= 761,IPUMIV1IV2['ind1990'] <= 781), 'ind'] = 9 #beauty shops\nIPUMIV1IV2.loc[IPUMIV1IV2['ind_int'] =

In [123]:
#*****some industries macth to multiple consumption goods
IPUMIV1IV2 = pd.read_stata('IPUMIV1IV2.dta', convert_categoricals=False)

IPUMIV1IV2_1 = IPUMIV1IV2[IPUMIV1IV2.ind != 5]
IPUMIV1IV2_1 = IPUMIV1IV2[IPUMIV1IV2.ind != 24]
IPUMIV1IV2_1 = IPUMIV1IV2[IPUMIV1IV2.ind != 28]

IPUMIV1IV2_1

,year,statefip,perwt,age,sex,race,citizen,ind1990,wkswork1,uhrswork,...,sharenoncollege,sharecollegeTR,sharenoncollegeTR,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1,ddtotedu_IVHLTR,ddtotedu_IVHLTR0,ind
0,1980,4,20,39,0,1,0,930,48,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1,1980,4,20,37,1,1,0,641,52,38,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2.0
2,1980,4,20,54,0,1,0,621,52,44,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
3,1980,4,20,20,1,1,0,861,52,40,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,37.0
4,1980,4,20,47,1,1,0,831,52,48,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8443109,2000,936,26,24,0,1,0,60,52,40,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,10.0
8443110,2000,936,11,24,1,1,0,722,48,40,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
8443111,2000,936,29,47,0,1,0,331,52,40,...,1.0,NaN,1.0,NaN,NaN,NaN,1.0,0.0,0.0,NaN
8443112,2000,936,22,37,0,1,0,910,52,40,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN


In [124]:
IPUMIV1IV2_1.to_stata('temp14')

In [125]:
IPUMIV1IV2_1.loc[IPUMIV1IV2_1['ind_int'] == 71, 'ind'] = 24
IPUMIV1IV2_1 = IPUMIV1IV2_1[IPUMIV1IV2_1.ind == 24]
IPUMIV1IV2_1

,year,statefip,perwt,age,sex,race,citizen,ind1990,wkswork1,uhrswork,...,sharenoncollege,sharecollegeTR,sharenoncollegeTR,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1,ddtotedu_IVHLTR,ddtotedu_IVHLTR0,ind
11,1980,4,20,53,0,1,0,712,52,40,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,24.0
39,1980,4,20,53,0,1,0,711,52,60,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,24.0
58,1980,4,20,53,0,1,0,710,50,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,24.0
68,1980,4,20,29,0,1,0,711,52,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,24.0
72,1980,4,20,34,1,1,0,711,48,40,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8442957,2000,936,13,49,1,1,0,712,52,45,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,24.0
8443065,2000,936,37,33,1,1,1,712,52,40,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,24.0
8443089,2000,936,32,43,1,3,0,711,52,40,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,24.0
8443102,2000,936,32,30,1,1,0,712,52,50,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,24.0


In [126]:
IPUMIV1IV2_1.to_stata('temp24')

In [127]:
temp14 = pd.read_stata('temp14')

In [128]:
temp14.loc[temp14['ind_int'] == 71, 'ind'] = 28
IPUMIV1IV2_2 = temp14[temp14.ind == 28]
IPUMIV1IV2_2

,index,year,statefip,perwt,age,sex,race,citizen,ind1990,wkswork1,...,sharenoncollege,sharecollegeTR,sharenoncollegeTR,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1,ddtotedu_IVHLTR,ddtotedu_IVHLTR0,ind
11,11,1980,4,20,53,0,1,0,712,52,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,28.0
39,39,1980,4,20,53,0,1,0,711,52,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,28.0
58,58,1980,4,20,53,0,1,0,710,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,28.0
68,68,1980,4,20,29,0,1,0,711,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,28.0
72,72,1980,4,20,34,1,1,0,711,48,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8442957,8442957,2000,936,13,49,1,1,0,712,52,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,28.0
8443065,8443065,2000,936,37,33,1,1,1,712,52,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,28.0
8443089,8443089,2000,936,32,43,1,3,0,711,52,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,28.0
8443102,8443102,2000,936,32,30,1,1,0,712,52,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,28.0


In [129]:
IPUMIV1IV2_2.to_stata('temp34')

In [130]:
temp14_2 = pd.read_stata('temp14')
temp14_2.loc[temp14_2['ind1990'] == 641, 'ind'] = 5
IPUMIV1IV2_3 = temp14_2[temp14_2.ind == 5]
IPUMIV1IV2_3

,index,year,statefip,perwt,age,sex,race,citizen,ind1990,wkswork1,...,sharenoncollege,sharecollegeTR,sharenoncollegeTR,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1,ddtotedu_IVHLTR,ddtotedu_IVHLTR0,ind
1,1,1980,4,20,37,1,1,0,641,52,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,5.0
28,28,1980,4,20,32,0,1,0,641,52,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,5.0
30,30,1980,4,20,27,1,1,0,641,50,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,5.0
54,54,1980,4,20,24,0,1,0,641,40,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,5.0
66,66,1980,4,20,18,0,1,0,641,48,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8442909,8442909,2000,936,21,19,1,7,1,641,52,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,5.0
8442946,8442946,2000,936,24,45,1,1,0,641,52,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,5.0
8442962,8442962,2000,936,27,52,1,1,0,641,52,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,5.0
8442990,8442990,2000,936,27,42,0,7,1,641,40,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,5.0


In [131]:
IPUMIV1IV2_3.to_stata('temp44')

In [132]:
temp111 = pd.read_stata('temp14')
temp1111 = temp111[temp111['ind'].notna()]

In [133]:
temp1111 = temp1111.fillna(0)
temp1111

,index,year,statefip,perwt,age,sex,race,citizen,ind1990,wkswork1,...,sharenoncollege,sharecollegeTR,sharenoncollegeTR,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1,ddtotedu_IVHLTR,ddtotedu_IVHLTR0,ind
1,1,1980,4,20,37,1,1,0,641,52,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3,1980,4,20,20,1,1,0,861,52,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0
4,4,1980,4,20,47,1,1,0,831,52,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0
5,5,1980,4,20,40,1,1,0,842,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0
7,7,1980,4,20,37,1,1,0,750,52,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8443102,8443102,2000,936,32,30,1,1,0,712,52,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0
8443103,8443103,2000,936,19,48,1,1,0,711,52,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0
8443108,8443108,2000,936,19,55,0,1,0,60,52,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
8443109,8443109,2000,936,26,24,0,1,0,60,52,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0


In [134]:
temp2 = pd.read_stata('temp24')
temp3 = pd.read_stata('temp34')
temp4 = pd.read_stata('temp44')

In [135]:
temp_joint = temp1111.append([temp2,temp3,temp4])

In [136]:
temp_joint = temp_joint.drop(columns=['ind_int', 'ind1990'])
temp_joint = temp_joint.fillna(0)
temp_joint

,index,year,statefip,perwt,age,sex,race,citizen,wkswork1,uhrswork,...,sharecollegeTR,sharenoncollegeTR,sharecollegeTR0,sharenoncollegeTR0,sharecollegeTR1,sharenoncollegeTR1,ddtotedu_IVHLTR,ddtotedu_IVHLTR0,ind,level_0
1,1,1980,4,20,37,1,1,0,52,38,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,3,1980,4,20,20,1,1,0,52,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,0.0
4,4,1980,4,20,47,1,1,0,52,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0
5,5,1980,4,20,40,1,1,0,40,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0
7,7,1980,4,20,37,1,1,0,52,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264622,8442909,2000,936,21,19,1,7,1,52,60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,8442909.0
264623,8442946,2000,936,24,45,1,1,0,52,32,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,8442946.0
264624,8442962,2000,936,27,52,1,1,0,52,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,8442962.0
264625,8442990,2000,936,27,42,0,7,1,40,50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,8442990.0


In [137]:
#*BALANCED SAMPLE OF MSAS, AGE MEAN EDUCATION ARE AT THE MSA LEVEL

In [138]:
temp_joint_1 = temp_joint.rename(columns={'sharecollegeTR':'IVTR', 'sharenoncollegeTR':'IV1TR', 'sharecollegeTR0':'IVTR0',
                                          'sharenoncollegeTR0':'IV1TR0', 'sharecollegeTR1':'IVTR1', 'sharenoncollegeTR1':'IV1TR1',
                                          'sharecollege':'IV', 'sharenoncollege':'IV1','college':'totempl'})

In [139]:
temp_joint_1

,index,year,statefip,perwt,age,sex,race,citizen,wkswork1,uhrswork,...,IVTR,IV1TR,IVTR0,IV1TR0,IVTR1,IV1TR1,ddtotedu_IVHLTR,ddtotedu_IVHLTR0,ind,level_0
1,1,1980,4,20,37,1,1,0,52,38,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,3,1980,4,20,20,1,1,0,52,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,0.0
4,4,1980,4,20,47,1,1,0,52,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0
5,5,1980,4,20,40,1,1,0,40,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0
7,7,1980,4,20,37,1,1,0,52,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264622,8442909,2000,936,21,19,1,7,1,52,60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,8442909.0
264623,8442946,2000,936,24,45,1,1,0,52,32,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,8442946.0
264624,8442962,2000,936,27,52,1,1,0,52,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,8442962.0
264625,8442990,2000,936,27,42,0,7,1,40,50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,8442990.0


In [140]:
temp_joint_2 = temp_joint_1.groupby(['statefip','year'])['IVTR','IV1TR','IVTR0','IV1TR0','IVTR1','IV1TR1','IV','IV1'].sum().sort_values(by=['statefip','year'])
temp_joint_2['totempl'] =temp_joint_2['IV'] + temp_joint_2['IV1'] 

temp_joint_3 = temp_joint_1.groupby(['statefip','year'])['age', 'sex', 'citizen'].mean().sort_values(by=['statefip','year'])
tjoint = pd.merge(temp_joint_2, temp_joint_3, on=['statefip','year'])
tjoint

<ipython-input-140-54708394aab2>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp_joint_2 = temp_joint_1.groupby(['statefip','year'])['IVTR','IV1TR','IVTR0','IV1TR0','IVTR1','IV1TR1','IV','IV1'].sum().sort_values(by=['statefip','year'])
<ipython-input-140-54708394aab2>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp_joint_3 = temp_joint_1.groupby(['statefip','year'])['age', 'sex', 'citizen'].mean().sort_values(by=['statefip','year'])


IVTR  IV1TR  IVTR0  IV1TR0  IVTR1  IV1TR1      IV     IV1  \
statefip year                                                               
4        1980   17.0   58.0   16.0    51.0    1.0     7.0   283.0   682.0   
         1990   14.0   64.0   12.0    62.0    2.0     2.0   329.0   796.0   
         2000   14.0   49.0   11.0    45.0    3.0     4.0   336.0   951.0   
8        1980   93.0  450.0   66.0   288.0   27.0   162.0  1414.0  3339.0   
         1990  109.0  525.0   74.0   314.0   35.0   211.0  1381.0  3616.0   
...              ...    ...    ...     ...    ...     ...     ...     ...   
932      2000  158.0  740.0   36.0   164.0  122.0   576.0  1223.0  3619.0   
934      1990    6.0   48.0    6.0    42.0    0.0     6.0   149.0   655.0   
         2000   12.0   78.0   12.0    74.0    0.0     4.0   191.0   830.0   
936      1990    2.0   17.0    2.0    12.0    0.0     5.0   134.0   463.0   
         2000    3.0   15.0    2.0    14.0    1.0     1.0   178.0   713.0   

               totempl        age       sex   citizen  
statefip year                                          
4        1980    965.0  37.158549  0.424870  0.021762  
         1990   1125.0  39.019556  0.512000  0.016889  
         2000   1287.0  38.476301  0.514375  0.025641  
8        1980   4753.0  37.176310  0.479907  0.009047  
         1990   4997.0  37.688213  0.496298  0.008805  
...                ...        ...       ...       ...  
932      2000   4842.0  40.566295  0.517555  0.005576  
934      1990    804.0  38.570896  0.513682  0.031095  
         2000   1021.0  40.546523  0.505387  0.064643  
936      1990    597.0  38.020101  0.477387  0.073702  
         2000    891.0  39.081930  0.524130  0.162738  

[787 rows x 12 columns]

In [141]:
tjoint['N'] = 1
tjoint['n'] = tjoint.groupby(['statefip'])['N'].transform('sum')
tjoint = tjoint.drop(columns=['N'])
tjoint = tjoint[tjoint.n == 3]
tjoint = tjoint.drop(columns=['n'])
tjoint

IVTR  IV1TR  IVTR0  IV1TR0  IVTR1  IV1TR1      IV     IV1  \
statefip year                                                               
4        1980   17.0   58.0   16.0    51.0    1.0     7.0   283.0   682.0   
         1990   14.0   64.0   12.0    62.0    2.0     2.0   329.0   796.0   
         2000   14.0   49.0   11.0    45.0    3.0     4.0   336.0   951.0   
8        1980   93.0  450.0   66.0   288.0   27.0   162.0  1414.0  3339.0   
         1990  109.0  525.0   74.0   314.0   35.0   211.0  1381.0  3616.0   
...              ...    ...    ...     ...    ...     ...     ...     ...   
928      1990   77.0  661.0   52.0   463.0   25.0   198.0   727.0  2801.0   
         2000   96.0  578.0   64.0   408.0   32.0   170.0   966.0  2736.0   
932      1980   70.0  648.0   20.0   129.0   50.0   519.0   832.0  2611.0   
         1990   64.0  704.0   10.0   155.0   54.0   549.0   787.0  2747.0   
         2000  158.0  740.0   36.0   164.0  122.0   576.0  1223.0  3619.0   

               totempl        age       sex   citizen  
statefip year                                          
4        1980    965.0  37.158549  0.424870  0.021762  
         1990   1125.0  39.019556  0.512000  0.016889  
         2000   1287.0  38.476301  0.514375  0.025641  
8        1980   4753.0  37.176310  0.479907  0.009047  
         1990   4997.0  37.688213  0.496298  0.008805  
...                ...        ...       ...       ...  
928      1990   3528.0  37.798186  0.478175  0.006803  
         2000   3702.0  40.195030  0.502971  0.015127  
932      1980   3443.0  38.070868  0.431600  0.011327  
         1990   3534.0  39.704867  0.490096  0.003396  
         2000   4842.0  40.566295  0.517555  0.005576  

[657 rows x 12 columns]

In [142]:
tjoint['H_HLTR0'] = np.log(tjoint['IVTR0']/tjoint['IV1TR0'])
tjoint['H_HLTR1'] = np.log(tjoint['IVTR1']/tjoint['IV1TR1'])
tjoint['H_HLTR'] = np.log(tjoint['IVTR']/tjoint['IV1TR'])
tjoint['H_HL'] = np.log(tjoint['IV']/tjoint['IV1'])
tjoint

C:\Users\gimat\anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


IVTR  IV1TR  IVTR0  IV1TR0  IVTR1  IV1TR1      IV     IV1  \
statefip year                                                               
4        1980   17.0   58.0   16.0    51.0    1.0     7.0   283.0   682.0   
         1990   14.0   64.0   12.0    62.0    2.0     2.0   329.0   796.0   
         2000   14.0   49.0   11.0    45.0    3.0     4.0   336.0   951.0   
8        1980   93.0  450.0   66.0   288.0   27.0   162.0  1414.0  3339.0   
         1990  109.0  525.0   74.0   314.0   35.0   211.0  1381.0  3616.0   
...              ...    ...    ...     ...    ...     ...     ...     ...   
928      1990   77.0  661.0   52.0   463.0   25.0   198.0   727.0  2801.0   
         2000   96.0  578.0   64.0   408.0   32.0   170.0   966.0  2736.0   
932      1980   70.0  648.0   20.0   129.0   50.0   519.0   832.0  2611.0   
         1990   64.0  704.0   10.0   155.0   54.0   549.0   787.0  2747.0   
         2000  158.0  740.0   36.0   164.0  122.0   576.0  1223.0  3619.0   

               totempl        age       sex   citizen   H_HLTR0   H_HLTR1  \
statefip year                                                               
4        1980    965.0  37.158549  0.424870  0.021762 -1.159237 -1.945910   
         1990   1125.0  39.019556  0.512000  0.016889 -1.642228  0.000000   
         2000   1287.0  38.476301  0.514375  0.025641 -1.408767 -0.287682   
8        1980   4753.0  37.176310  0.479907  0.009047 -1.473306 -1.791759   
         1990   4997.0  37.688213  0.496298  0.008805 -1.445328 -1.796510   
...                ...        ...       ...       ...       ...       ...   
928      1990   3528.0  37.798186  0.478175  0.006803 -2.186483 -2.069391   
         2000   3702.0  40.195030  0.502971  0.015127 -1.852384 -1.670063   
932      1980   3443.0  38.070868  0.431600  0.011327 -1.864080 -2.339881   
         1990   3534.0  39.704867  0.490096  0.003396 -2.740840 -2.319114   
         2000   4842.0  40.566295  0.517555  0.005576 -1.516348 -1.552087   

                 H_HLTR      H_HL  
statefip year                      
4        1980 -1.227230 -0.879583  
         1990 -1.519826 -0.883541  
         2000 -1.252763 -1.040403  
8        1980 -1.576648 -0.859249  
         1990 -1.572050 -0.962561  
...                 ...       ...  
928      1990 -2.149948 -1.348805  
         2000 -1.795226 -1.041088  
932      1980 -2.225395 -1.143656  
         1990 -2.397895 -1.250036  
         2000 -1.544055 -1.084891  

[657 rows x 16 columns]

In [143]:
tjoint1 = tjoint.drop(columns=['IV', 'IV1', 'IVTR', 'IV1TR', 'IVTR1', 'IV1TR1', 'IVTR0', 'IV1TR0'])
tjoint1

totempl        age       sex   citizen   H_HLTR0   H_HLTR1  \
statefip year                                                               
4        1980    965.0  37.158549  0.424870  0.021762 -1.159237 -1.945910   
         1990   1125.0  39.019556  0.512000  0.016889 -1.642228  0.000000   
         2000   1287.0  38.476301  0.514375  0.025641 -1.408767 -0.287682   
8        1980   4753.0  37.176310  0.479907  0.009047 -1.473306 -1.791759   
         1990   4997.0  37.688213  0.496298  0.008805 -1.445328 -1.796510   
...                ...        ...       ...       ...       ...       ...   
928      1990   3528.0  37.798186  0.478175  0.006803 -2.186483 -2.069391   
         2000   3702.0  40.195030  0.502971  0.015127 -1.852384 -1.670063   
932      1980   3443.0  38.070868  0.431600  0.011327 -1.864080 -2.339881   
         1990   3534.0  39.704867  0.490096  0.003396 -2.740840 -2.319114   
         2000   4842.0  40.566295  0.517555  0.005576 -1.516348 -1.552087   

                 H_HLTR      H_HL  
statefip year                      
4        1980 -1.227230 -0.879583  
         1990 -1.519826 -0.883541  
         2000 -1.252763 -1.040403  
8        1980 -1.576648 -0.859249  
         1990 -1.572050 -0.962561  
...                 ...       ...  
928      1990 -2.149948 -1.348805  
         2000 -1.795226 -1.041088  
932      1980 -2.225395 -1.143656  
         1990 -2.397895 -1.250036  
         2000 -1.544055 -1.084891  

[657 rows x 8 columns]

In [144]:
#Summary statistics
tjoint1[['totempl', 'age', 'sex', 'citizen', 'H_HLTR0', 'H_HLTR1', 'H_HLTR','H_HL']].describe()

,totempl,age,sex,citizen,H_HLTR0,H_HLTR1,H_HLTR,H_HL
count,657.000000,657.000000,657.000000,657.000000,657.000000,655.000000,657.000000,657.000000
mean,7612.587402,38.512168,0.486619,0.031215,-inf,-inf,-inf,-0.918776
std,15941.054688,1.312405,0.037370,0.037526,NaN,NaN,NaN,0.289837
min,482.000000,34.576923,0.346063,0.000000,-inf,-inf,-inf,-1.730333
25%,1537.000000,37.591865,0.463995,0.009059,-1.867267,-2.220410,-1.961051,-1.109955
50%,2950.000000,38.426938,0.489207,0.017704,-1.481604,-1.719786,-1.547880,-0.931451
75%,6956.000000,39.506695,0.512396,0.035471,-1.161925,-1.227960,-1.192877,-0.754948
max,184512.000000,41.900524,0.584388,0.283210,0.754841,0.251314,0.671168,0.626772


In [145]:
tjoint1.to_stata('tmmp_4')

In [146]:
#before in merge was temp_joint_11 instead of temp_joint
tjoint1_new  = pd.merge(temp_joint, tjoint1, on=['statefip','year'])
tjoint1_new

,index,year,statefip,perwt,age_x,sex_x,race,citizen_x,wkswork1,uhrswork,...,ind,level_0,totempl,age_y,sex_y,citizen_y,H_HLTR0,H_HLTR1,H_HLTR,H_HL
0,1,1980,4,20,37,1,1,0,52,38,...,2.0,0.0,965.0,37.158549,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583
1,3,1980,4,20,20,1,1,0,52,40,...,37.0,0.0,965.0,37.158549,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583
2,4,1980,4,20,47,1,1,0,52,48,...,21.0,0.0,965.0,37.158549,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583
3,5,1980,4,20,40,1,1,0,40,40,...,36.0,0.0,965.0,37.158549,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583
4,7,1980,4,20,37,1,1,0,52,40,...,26.0,0.0,965.0,37.158549,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5001465,8436338,2000,932,16,18,0,1,0,52,40,...,5.0,8436338.0,4842.0,40.566295,0.517555,0.005576,-1.516348,-1.552087,-1.544055,-1.084891
5001466,8436344,2000,932,25,49,1,1,0,49,30,...,5.0,8436344.0,4842.0,40.566295,0.517555,0.005576,-1.516348,-1.552087,-1.544055,-1.084891
5001467,8436351,2000,932,19,42,0,1,0,52,55,...,5.0,8436351.0,4842.0,40.566295,0.517555,0.005576,-1.516348,-1.552087,-1.544055,-1.084891
5001468,8436364,2000,932,28,19,0,1,0,52,30,...,5.0,8436364.0,4842.0,40.566295,0.517555,0.005576,-1.516348,-1.552087,-1.544055,-1.084891


In [147]:
#***at THIS POINT I HAVE TO COLLAPSE BY (STATE INDUSTRY AND YEAR) A I HAVE FINAL DATASET

In [148]:
tjoint1_new['empl0'] = 1
tjoint1_new['empl'] = tjoint1_new.groupby(['statefip', 'ind', 'year'])['empl0'].transform('sum')
tjoint1_new = tjoint1_new.sort_values(by=['statefip', 'ind', 'year']).drop(columns=['empl0'])
tjoint1_new

,index,year,statefip,perwt,age_x,sex_x,race,citizen_x,wkswork1,uhrswork,...,level_0,totempl,age_y,sex_y,citizen_y,H_HLTR0,H_HLTR1,H_HLTR,H_HL,empl
62,127,1980,4,20,27,0,1,0,52,40,...,0.0,965.0,37.158549,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583,25
68,138,1980,4,20,49,0,1,0,52,40,...,0.0,965.0,37.158549,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583,25
143,288,1980,4,20,56,0,1,1,52,47,...,0.0,965.0,37.158549,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583,25
151,304,1980,4,20,51,0,1,0,52,44,...,0.0,965.0,37.158549,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583,25
191,404,1980,4,20,19,0,1,0,52,40,...,0.0,965.0,37.158549,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5000399,8435773,2000,932,9,47,0,1,0,52,40,...,0.0,4842.0,40.566295,0.517555,0.005576,-1.516348,-1.552087,-1.544055,-1.084891,163
5000402,8435780,2000,932,26,38,0,1,0,52,50,...,0.0,4842.0,40.566295,0.517555,0.005576,-1.516348,-1.552087,-1.544055,-1.084891,163
5000507,8436024,2000,932,26,38,1,2,0,50,32,...,0.0,4842.0,40.566295,0.517555,0.005576,-1.516348,-1.552087,-1.544055,-1.084891,163
5000638,8436304,2000,932,22,52,1,1,0,52,40,...,0.0,4842.0,40.566295,0.517555,0.005576,-1.516348,-1.552087,-1.544055,-1.084891,163


In [149]:
tjoint1_new['nontrade'] = 1
tjoint1_new.loc[np.logical_or(tjoint1_new['ind'] == 1,tjoint1_new['ind'] == 3), 'nontrade'] = 0

In [150]:
tjoint1_new.loc[np.logical_or(tjoint1_new['ind'] == 4,tjoint1_new['ind'] == 6), 'nontrade'] = 0
tjoint1_new.loc[np.logical_or(tjoint1_new['ind'] == 8,tjoint1_new['ind'] == 11), 'nontrade'] = 0

In [151]:
tjoint1_new.loc[np.logical_or(tjoint1_new['ind'] == 15,tjoint1_new['ind'] == 18), 'nontrade'] = 0
tjoint1_new.loc[np.logical_or(tjoint1_new['ind'] == 19,tjoint1_new['ind'] == 25), 'nontrade'] = 0

In [152]:
tjoint1_new.loc[tjoint1_new['ind'] == 27, 'nontrade'] = 0
tjoint1_new.loc[np.logical_and(tjoint1_new['ind'] >= 32,tjoint1_new['ind'] <= 33), 'nontrade'] = 0
tjoint1_new

,index,year,statefip,perwt,age_x,sex_x,race,citizen_x,wkswork1,uhrswork,...,totempl,age_y,sex_y,citizen_y,H_HLTR0,H_HLTR1,H_HLTR,H_HL,empl,nontrade
62,127,1980,4,20,27,0,1,0,52,40,...,965.0,37.158549,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583,25,0
68,138,1980,4,20,49,0,1,0,52,40,...,965.0,37.158549,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583,25,0
143,288,1980,4,20,56,0,1,1,52,47,...,965.0,37.158549,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583,25,0
151,304,1980,4,20,51,0,1,0,52,44,...,965.0,37.158549,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583,25,0
191,404,1980,4,20,19,0,1,0,52,40,...,965.0,37.158549,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583,25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5000399,8435773,2000,932,9,47,0,1,0,52,40,...,4842.0,40.566295,0.517555,0.005576,-1.516348,-1.552087,-1.544055,-1.084891,163,1
5000402,8435780,2000,932,26,38,0,1,0,52,50,...,4842.0,40.566295,0.517555,0.005576,-1.516348,-1.552087,-1.544055,-1.084891,163,1
5000507,8436024,2000,932,26,38,1,2,0,50,32,...,4842.0,40.566295,0.517555,0.005576,-1.516348,-1.552087,-1.544055,-1.084891,163,1
5000638,8436304,2000,932,22,52,1,1,0,52,40,...,4842.0,40.566295,0.517555,0.005576,-1.516348,-1.552087,-1.544055,-1.084891,163,1


In [153]:
tjoint1_new['templ0'] = 1
tjoint1_new['templ'] = tjoint1_new.groupby(['statefip', 'nontrade', 'year'])['templ0'].transform('sum')
tjoint1_new = tjoint1_new.sort_values(by=['statefip', 'nontrade', 'year']).drop(columns=['templ0'])
tjoint1_new.loc[tjoint1_new['nontrade'] == 1, 'totemplNTR'] = tjoint1_new['templ']
tjoint1_new

,index,year,statefip,perwt,age_x,sex_x,race,citizen_x,wkswork1,uhrswork,...,sex_y,citizen_y,H_HLTR0,H_HLTR1,H_HLTR,H_HL,empl,nontrade,templ,totemplNTR
62,127,1980,4,20,27,0,1,0,52,40,...,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583,25,0,186,NaN
68,138,1980,4,20,49,0,1,0,52,40,...,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583,25,0,186,NaN
143,288,1980,4,20,56,0,1,1,52,47,...,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583,25,0,186,NaN
151,304,1980,4,20,51,0,1,0,52,44,...,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583,25,0,186,NaN
191,404,1980,4,20,19,0,1,0,52,40,...,0.424870,0.021762,-1.159237,-1.945910,-1.227230,-0.879583,25,0,186,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5000399,8435773,2000,932,9,47,0,1,0,52,40,...,0.517555,0.005576,-1.516348,-1.552087,-1.544055,-1.084891,163,1,3925,3925.0
5000402,8435780,2000,932,26,38,0,1,0,52,50,...,0.517555,0.005576,-1.516348,-1.552087,-1.544055,-1.084891,163,1,3925,3925.0
5000507,8436024,2000,932,26,38,1,2,0,50,32,...,0.517555,0.005576,-1.516348,-1.552087,-1.544055,-1.084891,163,1,3925,3925.0
5000638,8436304,2000,932,22,52,1,1,0,52,40,...,0.517555,0.005576,-1.516348,-1.552087,-1.544055,-1.084891,163,1,3925,3925.0


In [154]:
tjoint1_new.columns

Index(['index', 'year', 'statefip', 'perwt', 'age_x', 'sex_x', 'race',
       'citizen_x', 'wkswork1', 'uhrswork', 'incwage', 'college',
       'sharecollege', 'sharenoncollege', 'sharecollegeTR',
       'sharenoncollegeTR', 'sharecollegeTR0', 'sharenoncollegeTR0',
       'sharecollegeTR1', 'sharenoncollegeTR1', 'ddtotedu_IVHLTR',
       'ddtotedu_IVHLTR0', 'ind', 'level_0', 'totempl', 'age_y', 'sex_y',
       'citizen_y', 'H_HLTR0', 'H_HLTR1', 'H_HLTR', 'H_HL', 'empl', 'nontrade',
       'templ', 'totemplNTR'],
      dtype='object')

In [155]:
tjoint1_new = tjoint1_new.drop(columns=['college', 'sharecollege', 'sharenoncollege','age_y','sex_y','citizen_y'])

In [156]:
tjoint1_new2 = tjoint1_new.groupby(['statefip', 'ind', 'year'])['H_HL', 'H_HLTR', 'H_HLTR0', 'H_HLTR1', 'sex_x', 'age_x', 'citizen_x', 'empl',
                                        'totemplNTR', 'totempl', 'ddtotedu_IVHLTR', 'ddtotedu_IVHLTR0'].mean()

<ipython-input-156-56aed0f0a5a8>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  tjoint1_new2 = tjoint1_new.groupby(['statefip', 'ind', 'year'])['H_HL', 'H_HLTR', 'H_HLTR0', 'H_HLTR1', 'sex_x', 'age_x', 'citizen_x', 'empl',


In [157]:
tjoint1_new2

H_HL    H_HLTR   H_HLTR0   H_HLTR1     sex_x  \
statefip ind  year                                                     
4        1.0  1980 -0.879583 -1.227230 -1.159237 -1.945910  0.120000   
              1990 -0.883541 -1.519826 -1.642228  0.000000  0.130435   
              2000 -1.040403 -1.252763 -1.408767 -0.287682  0.300000   
         2.0  1980 -0.879583 -1.227230 -1.159237 -1.945910  0.684211   
              1990 -0.883541 -1.519826 -1.642228  0.000000  0.590164   
...                      ...       ...       ...       ...       ...   
932      37.0 1990 -1.250036 -2.397895 -2.740840 -2.319114  0.652174   
              2000 -1.084891 -1.544055 -1.516348 -1.552087  0.750000   
         38.0 1980 -1.143656 -2.225395 -1.864080 -2.339881  0.522124   
              1990 -1.250036 -2.397895 -2.740840 -2.319114  0.626168   
              2000 -1.084891 -1.544055 -1.516348 -1.552087  0.564417   

                        age_x  citizen_x  empl  totemplNTR  totempl  \
statefip ind  year                                                    
4        1.0  1980  40.360000   0.080000    25         NaN    965.0   
              1990  37.956522   0.130435    23         NaN   1125.0   
              2000  42.200000   0.150000    20         NaN   1287.0   
         2.0  1980  34.473684   0.078947    38       779.0    965.0   
              1990  33.950820   0.000000    61       988.0   1125.0   
...                       ...        ...   ...         ...      ...   
932      37.0 1990  39.304348   0.000000    23      2744.0   3534.0   
              2000  43.884615   0.038462    52      3925.0   4842.0   
         38.0 1980  41.176991   0.000000   113      2701.0   3443.0   
              1990  42.925234   0.000000   107      2744.0   3534.0   
              2000  44.515337   0.006135   163      3925.0   4842.0   

                    ddtotedu_IVHLTR  ddtotedu_IVHLTR0  
statefip ind  year                                     
4        1.0  1980         0.000000          0.000000  
              1990         0.082779          0.085779  
              2000         0.180968          0.211425  
         2.0  1980         0.000000          0.000000  
              1990         0.082779          0.085779  
...                             ...               ...  
932      37.0 1990         0.060024          0.093382  
              2000         0.131744          0.248677  
         38.0 1980         0.000000          0.000000  
              1990         0.060024          0.093382  
              2000         0.131744          0.248677  

[23264 rows x 12 columns]

In [158]:
tjoint1_new2 = tjoint1_new2.rename(columns={'sex_x':'sex', 'age_x':'age', 'citizen_x':'citizen','totempl_y':'totempl'})
tjoint1_new2 = tjoint1_new2.reset_index('year')
tjoint1_new2 = tjoint1_new2.reset_index('statefip')
tjoint1_new2

,statefip,year,H_HL,H_HLTR,H_HLTR0,H_HLTR1,sex,age,citizen,empl,totemplNTR,totempl,ddtotedu_IVHLTR,ddtotedu_IVHLTR0
ind,,,,,,,,,,,,,,
1.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.120000,40.360000,0.080000,25,NaN,965.0,0.000000,0.000000
1.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.130435,37.956522,0.130435,23,NaN,1125.0,0.082779,0.085779
1.0,4,2000,-1.040403,-1.252763,-1.408767,-0.287682,0.300000,42.200000,0.150000,20,NaN,1287.0,0.180968,0.211425
2.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.684211,34.473684,0.078947,38,779.0,965.0,0.000000,0.000000
2.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.590164,33.950820,0.000000,61,988.0,1125.0,0.082779,0.085779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.652174,39.304348,0.000000,23,2744.0,3534.0,0.060024,0.093382
37.0,932,2000,-1.084891,-1.544055,-1.516348,-1.552087,0.750000,43.884615,0.038462,52,3925.0,4842.0,0.131744,0.248677
38.0,932,1980,-1.143656,-2.225395,-1.864080,-2.339881,0.522124,41.176991,0.000000,113,2701.0,3443.0,0.000000,0.000000


In [159]:
#*********merge with education elasticities 1994-1997 produced by figure1_table1&2.do
elast_IPUM = pd.read_stata('elast_IPUM')
IPUMelast = pd.merge(tjoint1_new2, elast_IPUM, on=['ind'])
IPUMelast

,ind,statefip,year,H_HL,H_HLTR,H_HLTR0,H_HLTR1,sex,age,citizen,...,index,elastedu,elastincome,ratio80,educbeta,incomebeta,sigmaedu,sigmaincome,meansh,sgdum
0,1.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.120000,40.360000,0.080000,...,0,-0.147344,0.740111,0.236263,-0.051537,-0.047423,0.003674,0.005294,0.182473,0.001285
1,1.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.130435,37.956522,0.130435,...,0,-0.147344,0.740111,0.236263,-0.051537,-0.047423,0.003674,0.005294,0.182473,0.001285
2,1.0,4,2000,-1.040403,-1.252763,-1.408767,-0.287682,0.300000,42.200000,0.150000,...,0,-0.147344,0.740111,0.236263,-0.051537,-0.047423,0.003674,0.005294,0.182473,0.001285
3,1.0,8,1980,-0.859249,-1.576648,-1.473306,-1.791759,0.182796,36.774194,0.010753,...,0,-0.147344,0.740111,0.236263,-0.051537,-0.047423,0.003674,0.005294,0.182473,0.001285
4,1.0,8,1990,-0.962561,-1.572050,-1.445328,-1.796510,0.377778,38.155556,0.022222,...,0,-0.147344,0.740111,0.236263,-0.051537,-0.047423,0.003674,0.005294,0.182473,0.001285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23259,33.0,928,1990,-1.348805,-2.149948,-2.186483,-2.069391,0.219048,38.971429,0.000000,...,32,0.184298,1.398311,0.410437,0.008639,0.009740,0.013542,0.021356,0.024454,0.000635
23260,33.0,928,2000,-1.041088,-1.795226,-1.852384,-1.670063,0.209877,43.493827,0.000000,...,32,0.184298,1.398311,0.410437,0.008639,0.009740,0.013542,0.021356,0.024454,0.000635
23261,33.0,932,1980,-1.143656,-2.225395,-1.864080,-2.339881,0.571429,30.000000,0.000000,...,32,0.184298,1.398311,0.410437,0.008639,0.009740,0.013542,0.021356,0.024454,0.000635
23262,33.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.500000,35.500000,0.000000,...,32,0.184298,1.398311,0.410437,0.008639,0.009740,0.013542,0.021356,0.024454,0.000635


In [160]:
IPUMelast['share_w'] = IPUMelast['empl']/IPUMelast['totempl']
IPUMelast['share_wNTR'] = IPUMelast['empl']/IPUMelast['totemplNTR']
IPUMelast = IPUMelast.sort_values(['ind','statefip'])
IPUMelast

,ind,statefip,year,H_HL,H_HLTR,H_HLTR0,H_HLTR1,sex,age,citizen,...,elastincome,ratio80,educbeta,incomebeta,sigmaedu,sigmaincome,meansh,sgdum,share_w,share_wNTR
0,1.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.120000,40.360000,0.080000,...,0.740111,0.236263,-0.051537,-0.047423,0.003674,0.005294,0.182473,0.001285,0.025907,NaN
1,1.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.130435,37.956522,0.130435,...,0.740111,0.236263,-0.051537,-0.047423,0.003674,0.005294,0.182473,0.001285,0.020444,NaN
2,1.0,4,2000,-1.040403,-1.252763,-1.408767,-0.287682,0.300000,42.200000,0.150000,...,0.740111,0.236263,-0.051537,-0.047423,0.003674,0.005294,0.182473,0.001285,0.015540,NaN
3,1.0,8,1980,-0.859249,-1.576648,-1.473306,-1.791759,0.182796,36.774194,0.010753,...,0.740111,0.236263,-0.051537,-0.047423,0.003674,0.005294,0.182473,0.001285,0.019567,NaN
4,1.0,8,1990,-0.962561,-1.572050,-1.445328,-1.796510,0.377778,38.155556,0.022222,...,0.740111,0.236263,-0.051537,-0.047423,0.003674,0.005294,0.182473,0.001285,0.018011,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22308,38.0,928,1990,-1.348805,-2.149948,-2.186483,-2.069391,0.489130,41.010870,0.000000,...,1.024418,0.590352,0.007074,0.000417,0.016726,0.005628,0.017097,0.000548,0.026077,0.033201
22309,38.0,928,2000,-1.041088,-1.795226,-1.852384,-1.670063,0.580357,43.339286,0.008929,...,1.024418,0.590352,0.007074,0.000417,0.016726,0.005628,0.017097,0.000548,0.030254,0.037246
22310,38.0,932,1980,-1.143656,-2.225395,-1.864080,-2.339881,0.522124,41.176991,0.000000,...,1.024418,0.590352,0.007074,0.000417,0.016726,0.005628,0.017097,0.000548,0.032820,0.041836
22311,38.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.626168,42.925234,0.000000,...,1.024418,0.590352,0.007074,0.000417,0.016726,0.005628,0.017097,0.000548,0.030277,0.038994


In [161]:
#*DO FIRST DIFFERENCES
IPUMelast1 = IPUMelast.sort_values(['statefip','ind'])

In [162]:
#LAG
IPUMelast1['H_HL_lag'] = IPUMelast1.groupby(['statefip','ind'])['H_HL'].shift(1)
IPUMelast1['H_HLTR_lag'] = IPUMelast1.groupby(['statefip','ind'])['H_HLTR'].shift(1)
IPUMelast1['H_HLTR0_lag'] = IPUMelast1.groupby(['statefip','ind'])['H_HLTR0'].shift(1)
IPUMelast1['sex_lag'] = IPUMelast1.groupby(['statefip','ind'])['sex'].shift(1)
IPUMelast1['age_lag'] = IPUMelast1.groupby(['statefip','ind'])['age'].shift(1)
IPUMelast1['citizen_lag'] = IPUMelast1.groupby(['statefip','ind'])['citizen'].shift(1)

IPUMelast1['dH_HL'] = IPUMelast1['H_HL'] - IPUMelast1['H_HL_lag']
IPUMelast1['dH_HLTR'] = IPUMelast1['H_HLTR']-IPUMelast1['H_HLTR_lag']
IPUMelast1['dH_HLTR0'] = IPUMelast1['H_HLTR0']-IPUMelast1['H_HLTR0_lag']
IPUMelast1['dsex'] = IPUMelast1['sex'] - IPUMelast1['sex_lag']
IPUMelast1['dage'] = IPUMelast1['age'] - IPUMelast1['age_lag']
IPUMelast1['dcitizen'] = IPUMelast1['citizen'] - IPUMelast1['citizen_lag']

In [163]:
IPUMelast1

,ind,statefip,year,H_HL,H_HLTR,H_HLTR0,H_HLTR1,sex,age,citizen,...,H_HLTR0_lag,sex_lag,age_lag,citizen_lag,dH_HL,dH_HLTR,dH_HLTR0,dsex,dage,dcitizen
0,1.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.120000,40.360000,0.080000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.130435,37.956522,0.130435,...,-1.159237,0.120000,40.360000,0.080000,-0.003959,-0.292596,-0.482991,0.010435,-2.403478,0.050435
2,1.0,4,2000,-1.040403,-1.252763,-1.408767,-0.287682,0.300000,42.200000,0.150000,...,-1.642228,0.130435,37.956522,0.130435,-0.156861,0.267063,0.233461,0.169565,4.243478,0.019565
657,2.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.684211,34.473684,0.078947,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
658,2.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.590164,33.950820,0.000000,...,-1.159237,0.684211,34.473684,0.078947,-0.003959,-0.292596,-0.482991,-0.094047,-0.522865,-0.078947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21654,37.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.652174,39.304348,0.000000,...,-1.864080,0.571429,37.928571,0.000000,-0.106380,-0.172500,-0.876760,0.080745,1.375776,0.000000
21655,37.0,932,2000,-1.084891,-1.544055,-1.516348,-1.552087,0.750000,43.884615,0.038462,...,-2.740840,0.652174,39.304348,0.000000,0.165146,0.853840,1.224492,0.097826,4.580268,0.038462
22310,38.0,932,1980,-1.143656,-2.225395,-1.864080,-2.339881,0.522124,41.176991,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22311,38.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.626168,42.925234,0.000000,...,-1.864080,0.522124,41.176991,0.000000,-0.106380,-0.172500,-0.876760,0.104044,1.748242,0.000000


In [164]:
#LAG
IPUMelast1['share_wNTR_lag'] = IPUMelast1.groupby(['statefip','ind'])['share_wNTR'].shift(1)
IPUMelast1['share_w_lag'] = IPUMelast1.groupby(['statefip','ind'])['share_w'].shift(1)

IPUMelast1['dshare_wNTR'] = IPUMelast1['share_wNTR'] - IPUMelast1['share_wNTR_lag']
IPUMelast1['dshare_w'] = IPUMelast1['share_w']-IPUMelast1['share_w_lag']
IPUMelast1

,ind,statefip,year,H_HL,H_HLTR,H_HLTR0,H_HLTR1,sex,age,citizen,...,dH_HL,dH_HLTR,dH_HLTR0,dsex,dage,dcitizen,share_wNTR_lag,share_w_lag,dshare_wNTR,dshare_w
0,1.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.120000,40.360000,0.080000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.130435,37.956522,0.130435,...,-0.003959,-0.292596,-0.482991,0.010435,-2.403478,0.050435,NaN,0.025907,NaN,-0.005462
2,1.0,4,2000,-1.040403,-1.252763,-1.408767,-0.287682,0.300000,42.200000,0.150000,...,-0.156861,0.267063,0.233461,0.169565,4.243478,0.019565,NaN,0.020444,NaN,-0.004904
657,2.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.684211,34.473684,0.078947,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
658,2.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.590164,33.950820,0.000000,...,-0.003959,-0.292596,-0.482991,-0.094047,-0.522865,-0.078947,0.048780,0.039378,0.012960,0.014844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21654,37.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.652174,39.304348,0.000000,...,-0.106380,-0.172500,-0.876760,0.080745,1.375776,0.000000,0.010367,0.008132,-0.001985,-0.001624
21655,37.0,932,2000,-1.084891,-1.544055,-1.516348,-1.552087,0.750000,43.884615,0.038462,...,0.165146,0.853840,1.224492,0.097826,4.580268,0.038462,0.008382,0.006508,0.004866,0.004231
22310,38.0,932,1980,-1.143656,-2.225395,-1.864080,-2.339881,0.522124,41.176991,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22311,38.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.626168,42.925234,0.000000,...,-0.106380,-0.172500,-0.876760,0.104044,1.748242,0.000000,0.041836,0.032820,-0.002842,-0.002543


In [165]:
IPUMelast1['nontrade'] = 1

IPUMelast1.loc[np.logical_or(IPUMelast1['ind'] == 1,IPUMelast1['ind'] == 3), 'nontrade'] = 0
IPUMelast1.loc[np.logical_or(IPUMelast1['ind'] == 4,IPUMelast1['ind'] == 6), 'nontrade'] = 0

In [166]:
IPUMelast1.loc[np.logical_or(IPUMelast1['ind'] == 8,IPUMelast1['ind'] == 11), 'nontrade'] = 0
IPUMelast1.loc[np.logical_or(IPUMelast1['ind'] == 15,IPUMelast1['ind'] == 18), 'nontrade'] = 0

In [167]:
IPUMelast1.loc[np.logical_or(IPUMelast1['ind'] == 19,IPUMelast1['ind'] == 25), 'nontrade'] = 0
IPUMelast1.loc[IPUMelast1['ind'] == 27, 'nontrade'] = 0
IPUMelast1.loc[np.logical_and(IPUMelast1['ind'] >= 32,IPUMelast1['ind'] <= 33), 'nontrade'] = 0
IPUMelast1

,ind,statefip,year,H_HL,H_HLTR,H_HLTR0,H_HLTR1,sex,age,citizen,...,dH_HLTR,dH_HLTR0,dsex,dage,dcitizen,share_wNTR_lag,share_w_lag,dshare_wNTR,dshare_w,nontrade
0,1.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.120000,40.360000,0.080000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.130435,37.956522,0.130435,...,-0.292596,-0.482991,0.010435,-2.403478,0.050435,NaN,0.025907,NaN,-0.005462,0
2,1.0,4,2000,-1.040403,-1.252763,-1.408767,-0.287682,0.300000,42.200000,0.150000,...,0.267063,0.233461,0.169565,4.243478,0.019565,NaN,0.020444,NaN,-0.004904,0
657,2.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.684211,34.473684,0.078947,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
658,2.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.590164,33.950820,0.000000,...,-0.292596,-0.482991,-0.094047,-0.522865,-0.078947,0.048780,0.039378,0.012960,0.014844,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21654,37.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.652174,39.304348,0.000000,...,-0.172500,-0.876760,0.080745,1.375776,0.000000,0.010367,0.008132,-0.001985,-0.001624,1
21655,37.0,932,2000,-1.084891,-1.544055,-1.516348,-1.552087,0.750000,43.884615,0.038462,...,0.853840,1.224492,0.097826,4.580268,0.038462,0.008382,0.006508,0.004866,0.004231,1
22310,38.0,932,1980,-1.143656,-2.225395,-1.864080,-2.339881,0.522124,41.176991,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
22311,38.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.626168,42.925234,0.000000,...,-0.172500,-0.876760,0.104044,1.748242,0.000000,0.041836,0.032820,-0.002842,-0.002543,1


In [168]:
IPUMelast1.loc[IPUMelast1['nontrade'] == 1, 'tr'] = 0

IPUMelast1.loc[np.logical_or(IPUMelast1['ind'] == 8,IPUMelast1['ind'] == 15), 'tr'] = 1
IPUMelast1.loc[np.logical_or(IPUMelast1['ind'] == 19,IPUMelast1['ind'] == 25), 'tr'] = 1
IPUMelast1.loc[np.logical_or(IPUMelast1['ind'] == 27,IPUMelast1['ind'] == 33), 'tr'] = 1

In [169]:
IPUMelast1['services'] = 0

IPUMelast1.loc[(IPUMelast1['nontrade'] == 1)&(IPUMelast1['ratio80'] <= 0.44)&(IPUMelast1['ratio80'] != np.nan), 'services'] = 1
IPUMelast1.loc[(IPUMelast1['nontrade'] == 1)&(IPUMelast1['ratio80'] > 0.44)&(IPUMelast1['ratio80'] != np.nan), 'services'] = 2

In [170]:
IPUMelast1= IPUMelast1[['ind', 'statefip', 'year', 'H_HL', 'H_HLTR', 'H_HLTR0', 'H_HLTR1','sex', 'age', 'citizen', 'empl', 'totemplNTR', 'totempl',
'ddtotedu_IVHLTR', 'ddtotedu_IVHLTR0','incomebeta','educbeta','sgdum','meansh','sigmaincome','sigmaedu','elastincome', 
'elastedu','ratio80', 'share_w', 'share_wNTR', 'dH_HL', 'dH_HLTR', 'dH_HLTR0','dsex', 'dage','dcitizen', 'dshare_wNTR', 
'dshare_w', 'nontrade','tr', 'services']]
IPUMelast1

,ind,statefip,year,H_HL,H_HLTR,H_HLTR0,H_HLTR1,sex,age,citizen,...,dH_HLTR,dH_HLTR0,dsex,dage,dcitizen,dshare_wNTR,dshare_w,nontrade,tr,services
0,1.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.120000,40.360000,0.080000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0
1,1.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.130435,37.956522,0.130435,...,-0.292596,-0.482991,0.010435,-2.403478,0.050435,NaN,-0.005462,0,NaN,0
2,1.0,4,2000,-1.040403,-1.252763,-1.408767,-0.287682,0.300000,42.200000,0.150000,...,0.267063,0.233461,0.169565,4.243478,0.019565,NaN,-0.004904,0,NaN,0
657,2.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.684211,34.473684,0.078947,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,1
658,2.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.590164,33.950820,0.000000,...,-0.292596,-0.482991,-0.094047,-0.522865,-0.078947,0.012960,0.014844,1,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21654,37.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.652174,39.304348,0.000000,...,-0.172500,-0.876760,0.080745,1.375776,0.000000,-0.001985,-0.001624,1,0.0,2
21655,37.0,932,2000,-1.084891,-1.544055,-1.516348,-1.552087,0.750000,43.884615,0.038462,...,0.853840,1.224492,0.097826,4.580268,0.038462,0.004866,0.004231,1,0.0,2
22310,38.0,932,1980,-1.143656,-2.225395,-1.864080,-2.339881,0.522124,41.176991,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,2
22311,38.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.626168,42.925234,0.000000,...,-0.172500,-0.876760,0.104044,1.748242,0.000000,-0.002842,-0.002543,1,0.0,2


In [171]:
#*********************DESCRIPTIVES
#Summary statistics
SUelastedu = pd.DataFrame(IPUMelast1['elastedu'].describe())
SUelastedu

,elastedu
count,23264.000000
mean,0.079328
std,0.178641
min,-0.415191
25%,-0.056339
50%,0.073944
75%,0.184298
max,0.495370


In [172]:
#Summary statistics
SUelastedu1 = pd.DataFrame(IPUMelast1[IPUMelast1.services != 0]['elastedu'].describe())
SUelastedu1

,elastedu
count,15814.000000
mean,0.113921
std,0.174946
min,-0.141120
25%,-0.039832
50%,0.090148
75%,0.211143
max,0.495370


In [173]:
IPUMelast1['highs'] = 0
IPUMelast1.loc[IPUMelast1['elastedu'] >=0.09, 'highs'] = 1
IPUMelast1

<ipython-input-173-c376ffe94400>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IPUMelast1['highs'] = 0
C:\Users\gimat\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,ind,statefip,year,H_HL,H_HLTR,H_HLTR0,H_HLTR1,sex,age,citizen,...,dH_HLTR0,dsex,dage,dcitizen,dshare_wNTR,dshare_w,nontrade,tr,services,highs
0,1.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.120000,40.360000,0.080000,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0
1,1.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.130435,37.956522,0.130435,...,-0.482991,0.010435,-2.403478,0.050435,NaN,-0.005462,0,NaN,0,0
2,1.0,4,2000,-1.040403,-1.252763,-1.408767,-0.287682,0.300000,42.200000,0.150000,...,0.233461,0.169565,4.243478,0.019565,NaN,-0.004904,0,NaN,0,0
657,2.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.684211,34.473684,0.078947,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,1,1
658,2.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.590164,33.950820,0.000000,...,-0.482991,-0.094047,-0.522865,-0.078947,0.012960,0.014844,1,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21654,37.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.652174,39.304348,0.000000,...,-0.876760,0.080745,1.375776,0.000000,-0.001985,-0.001624,1,0.0,2,1
21655,37.0,932,2000,-1.084891,-1.544055,-1.516348,-1.552087,0.750000,43.884615,0.038462,...,1.224492,0.097826,4.580268,0.038462,0.004866,0.004231,1,0.0,2,1
22310,38.0,932,1980,-1.143656,-2.225395,-1.864080,-2.339881,0.522124,41.176991,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,2,1
22311,38.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.626168,42.925234,0.000000,...,-0.876760,0.104044,1.748242,0.000000,-0.002842,-0.002543,1,0.0,2,1


In [174]:
IPUMelast1.loc[IPUMelast1['year'] == 1980, 'age80'] = IPUMelast1['age']
IPUMelast1['age80_'] = IPUMelast1.groupby('statefip')['age80'].transform('mean')

C:\Users\gimat\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\gimat\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
<ipython-input-174-5f12c8c97bb4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [175]:
IPUMelast1.loc[IPUMelast1['year'] == 1980, 'sex80'] = IPUMelast1['sex']
IPUMelast1['sex80_'] = IPUMelast1.groupby('statefip')['sex80'].transform('mean')

C:\Users\gimat\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\gimat\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
<ipython-input-175-5583f99595e6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [176]:
IPUMelast1.loc[IPUMelast1['year'] == 1980, 'citizen80'] = IPUMelast1['citizen']
IPUMelast1['citizen80_'] = IPUMelast1.groupby('statefip')['citizen80'].transform('mean')

C:\Users\gimat\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\gimat\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
<ipython-input-176-613afa6de5da>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [177]:
IPUMelast1.loc[IPUMelast1['year'] == 1980, 'share80'] = IPUMelast1['share_w']
IPUMelast1['share80_'] = IPUMelast1.groupby('statefip')['share80'].transform('mean')

C:\Users\gimat\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\gimat\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [178]:
#LAG2
IPUMelast1['share_w_lag2'] = IPUMelast1.groupby(['statefip','ind'])['share_w'].shift(2)
IPUMelast1['ddshare'] = IPUMelast1['share_w']-IPUMelast1['share_w_lag2']

In [179]:
IPUMelast1['share_wNTR_lag2'] = IPUMelast1.groupby(['statefip','ind'])['share_wNTR'].shift(2)
IPUMelast1['ddshareNTR'] = IPUMelast1['share_wNTR']-IPUMelast1['share_wNTR_lag2']

IPUMelast1['dshareNTR_'] = IPUMelast1.groupby(['statefip','ind'])['ddshareNTR'].transform('mean')

In [180]:
IPUMelast1['share_w_lag'] = IPUMelast1.groupby(['statefip','ind'])['share_w'].shift(1)
IPUMelast1['dshare'] = IPUMelast1['share_w']-IPUMelast1['share_w_lag']
IPUMelast1.loc[IPUMelast1['year'] == 1990, 'dshare90'] = IPUMelast1['dshare']

IPUMelast1['dshare90_'] = IPUMelast1.groupby(['statefip','ind'])['dshare90'].transform('mean')

In [181]:
IPUMelast1.loc[IPUMelast1['year'] == 2000, 'dshare00'] = IPUMelast1['dshare']
IPUMelast1['dshare00_'] = IPUMelast1.groupby(['statefip','ind'])['dshare00'].transform('mean')

In [182]:
IPUMelast1 = IPUMelast1.drop(columns=['share_w_lag2','share_wNTR_lag2','share_w_lag'])
IPUMelast1

,ind,statefip,year,H_HL,H_HLTR,H_HLTR0,H_HLTR1,sex,age,citizen,...,share80,share80_,ddshare,ddshareNTR,dshareNTR_,dshare,dshare90,dshare90_,dshare00,dshare00_
0,1.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.120000,40.360000,0.080000,...,0.025907,0.029412,NaN,NaN,NaN,NaN,NaN,-0.005462,NaN,-0.004904
1,1.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.130435,37.956522,0.130435,...,NaN,0.029412,NaN,NaN,NaN,-0.005462,-0.005462,-0.005462,NaN,-0.004904
2,1.0,4,2000,-1.040403,-1.252763,-1.408767,-0.287682,0.300000,42.200000,0.150000,...,NaN,0.029412,-0.010367,NaN,NaN,-0.004904,NaN,-0.005462,-0.004904,-0.004904
657,2.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.684211,34.473684,0.078947,...,0.039378,0.029412,NaN,NaN,0.017384,NaN,NaN,0.014844,NaN,0.007161
658,2.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.590164,33.950820,0.000000,...,NaN,0.029412,NaN,NaN,0.017384,0.014844,0.014844,0.014844,NaN,0.007161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21654,37.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.652174,39.304348,0.000000,...,NaN,0.028571,NaN,NaN,0.002882,-0.001624,-0.001624,-0.001624,NaN,0.004231
21655,37.0,932,2000,-1.084891,-1.544055,-1.516348,-1.552087,0.750000,43.884615,0.038462,...,NaN,0.028571,0.002607,0.002882,0.002882,0.004231,NaN,-0.001624,0.004231,0.004231
22310,38.0,932,1980,-1.143656,-2.225395,-1.864080,-2.339881,0.522124,41.176991,0.000000,...,0.032820,0.028571,NaN,NaN,-0.000308,NaN,NaN,-0.002543,NaN,0.003386
22311,38.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.626168,42.925234,0.000000,...,NaN,0.028571,NaN,NaN,-0.000308,-0.002543,-0.002543,-0.002543,NaN,0.003386


In [183]:
IPUMelast1.loc[IPUMelast1['year'] == 1980, 'empl80'] = IPUMelast1['empl']
IPUMelast1['empl80_'] = IPUMelast1.groupby(['statefip','ind'])['empl80'].transform('mean')
IPUMelast1

,ind,statefip,year,H_HL,H_HLTR,H_HLTR0,H_HLTR1,sex,age,citizen,...,ddshare,ddshareNTR,dshareNTR_,dshare,dshare90,dshare90_,dshare00,dshare00_,empl80,empl80_
0,1.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.120000,40.360000,0.080000,...,NaN,NaN,NaN,NaN,NaN,-0.005462,NaN,-0.004904,25.0,25.0
1,1.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.130435,37.956522,0.130435,...,NaN,NaN,NaN,-0.005462,-0.005462,-0.005462,NaN,-0.004904,NaN,25.0
2,1.0,4,2000,-1.040403,-1.252763,-1.408767,-0.287682,0.300000,42.200000,0.150000,...,-0.010367,NaN,NaN,-0.004904,NaN,-0.005462,-0.004904,-0.004904,NaN,25.0
657,2.0,4,1980,-0.879583,-1.227230,-1.159237,-1.945910,0.684211,34.473684,0.078947,...,NaN,NaN,0.017384,NaN,NaN,0.014844,NaN,0.007161,38.0,38.0
658,2.0,4,1990,-0.883541,-1.519826,-1.642228,0.000000,0.590164,33.950820,0.000000,...,NaN,NaN,0.017384,0.014844,0.014844,0.014844,NaN,0.007161,NaN,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21654,37.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.652174,39.304348,0.000000,...,NaN,NaN,0.002882,-0.001624,-0.001624,-0.001624,NaN,0.004231,NaN,28.0
21655,37.0,932,2000,-1.084891,-1.544055,-1.516348,-1.552087,0.750000,43.884615,0.038462,...,0.002607,0.002882,0.002882,0.004231,NaN,-0.001624,0.004231,0.004231,NaN,28.0
22310,38.0,932,1980,-1.143656,-2.225395,-1.864080,-2.339881,0.522124,41.176991,0.000000,...,NaN,NaN,-0.000308,NaN,NaN,-0.002543,NaN,0.003386,113.0,113.0
22311,38.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.626168,42.925234,0.000000,...,NaN,NaN,-0.000308,-0.002543,-0.002543,-0.002543,NaN,0.003386,NaN,113.0


In [186]:
IPUMelast2 = IPUMelast1[IPUMelast1['empl80_']>=200]
IPUMelast2 = IPUMelast2[IPUMelast2['empl80_'] != np.nan]
IPUMelast2 

,ind,statefip,year,H_HL,H_HLTR,H_HLTR0,H_HLTR1,sex,age,citizen,...,ddshare,ddshareNTR,dshareNTR_,dshare,dshare90,dshare90_,dshare00,dshare00_,empl80,empl80_
660,2.0,8,1980,-0.859249,-1.576648,-1.473306,-1.791759,0.695205,33.684932,0.013699,...,NaN,NaN,0.009800,NaN,NaN,0.007807,NaN,0.000472,292.0,292.0
661,2.0,8,1990,-0.962561,-1.572050,-1.445328,-1.796510,0.604046,32.806358,0.020231,...,NaN,NaN,0.009800,0.007807,0.007807,0.007807,NaN,0.000472,NaN,292.0
662,2.0,8,2000,-0.770628,-1.350955,-1.190986,-1.589635,0.530660,33.266509,0.009434,...,0.008279,0.009800,0.009800,0.000472,NaN,0.007807,0.000472,0.000472,NaN,292.0
1944,5.0,8,1980,-0.859249,-1.576648,-1.473306,-1.791759,0.695205,33.684932,0.013699,...,NaN,NaN,0.009800,NaN,NaN,0.007807,NaN,0.000472,292.0,292.0
1945,5.0,8,1990,-0.962561,-1.572050,-1.445328,-1.796510,0.604046,32.806358,0.020231,...,NaN,NaN,0.009800,0.007807,0.007807,0.007807,NaN,0.000472,NaN,292.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14691,25.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.200000,43.843103,0.003448,...,NaN,NaN,NaN,0.004956,0.004956,0.004956,NaN,-0.027606,NaN,548.0
14692,25.0,932,2000,-1.084891,-1.544055,-1.516348,-1.552087,0.258699,45.444781,0.007564,...,-0.022650,NaN,NaN,-0.027606,NaN,0.004956,-0.027606,-0.027606,NaN,548.0
20996,36.0,932,1980,-1.143656,-2.225395,-1.864080,-2.339881,0.566489,40.364362,0.005319,...,NaN,NaN,-0.008762,NaN,NaN,0.001149,NaN,-0.004615,376.0,376.0
20997,36.0,932,1990,-1.250036,-2.397895,-2.740840,-2.319114,0.630769,43.174359,0.000000,...,NaN,NaN,-0.008762,0.001149,0.001149,0.001149,NaN,-0.004615,NaN,376.0


In [187]:
#*TABLE DESCRIPTIVES
#Summary statistics
pd.DataFrame(IPUMelast2[IPUMelast2.year == 1980][['sex', 'age', 'citizen', 'H_HLTR', 'H_HLTR0']].describe())

,sex,age,citizen,H_HLTR,H_HLTR0
count,1377.000000,1377.000000,1377.000000,1377.000000,1377.000000
mean,0.456773,37.534356,0.032754,-1.504410,-1.472730
std,0.207756,2.511778,0.050707,0.390697,0.396323
min,0.040909,29.109170,0.000000,-3.465736,-4.127134
25%,0.324723,36.185796,0.006570,-1.711853,-1.702664
50%,0.496045,37.688482,0.016327,-1.500487,-1.470176
75%,0.603306,39.166667,0.037500,-1.245262,-1.228252
max,0.888889,45.239714,0.579869,-0.784363,-0.528525


In [188]:
pd.DataFrame(IPUMelast2[IPUMelast2.year == 1990][['dH_HLTR', 'dH_HLTR0']].describe())

,dH_HLTR,dH_HLTR0
count,1377.000000,1377.000000
mean,0.158827,0.173587
std,0.255277,0.286621
min,-1.386294,-1.468532
25%,0.004598,0.027978
50%,0.204057,0.203339
75%,0.294230,0.318744
max,1.073202,1.543137


In [189]:
pd.DataFrame(IPUMelast2[IPUMelast2.year == 2000][['dH_HLTR', 'dH_HLTR0']].describe())

,dH_HLTR,dH_HLTR0
count,1376.000000,1376.000000
mean,0.341979,0.330756
std,0.219977,0.236558
min,-0.570070,-0.617640
25%,0.232378,0.223382
50%,0.353223,0.339035
75%,0.474991,0.465083
max,1.418019,1.658228


In [190]:
pd.DataFrame(IPUMelast2[(IPUMelast2.year == 1980) & (IPUMelast2.services == 2)][['empl80_', 'share80_', 'dshare90_', 'dshare00_', 'dshareNTR_']].describe())

,empl80_,share80_,dshare90_,dshare00_,dshareNTR_
count,765.000000,765.000000,765.000000,765.000000,765.000000
mean,858.975163,0.026985,0.000634,-0.001777,-0.005741
std,1417.245805,0.000964,0.011290,0.011683,0.016513
min,200.000000,0.026316,-0.080813,-0.076703,-0.080876
25%,294.000000,0.026316,-0.005019,-0.007343,-0.014193
50%,445.000000,0.026316,0.001670,-0.000848,-0.003444
75%,851.000000,0.027778,0.007720,0.004490,0.004314
max,14577.000000,0.033333,0.042713,0.052146,0.062359


In [191]:
pd.DataFrame(IPUMelast2[(IPUMelast2.year == 1980) & (IPUMelast2.services == 1)][['empl80_', 'share80_', 'dshare90_', 'dshare00_', 'dshareNTR_']].describe())

,empl80_,share80_,dshare90_,dshare00_,dshareNTR_
count,397.000000,397.000000,397.000000,397.000000,397.000000
mean,720.105793,0.026882,-0.001503,0.002945,-0.001743
std,921.111962,0.000923,0.020122,0.013777,0.022794
min,200.000000,0.026316,-0.214636,-0.081171,-0.178702
25%,272.000000,0.026316,-0.005177,-0.002510,-0.007307
50%,414.000000,0.026316,0.000861,0.002370,0.001987
75%,774.000000,0.027027,0.007996,0.008165,0.010269
max,7412.000000,0.035714,0.056506,0.062694,0.046789


In [192]:
pd.DataFrame(IPUMelast2[(IPUMelast2.year == 1980) & (IPUMelast2.services == 0)][['empl80_', 'share80_', 'dshare90_', 'dshare00_', 'dshareNTR_']].describe())

,empl80_,share80_,dshare90_,dshare00_,dshareNTR_
count,215.000000,215.000000,215.000000,214.000000,0.0
mean,691.804651,0.026757,-0.016819,-0.006544,NaN
std,1112.165224,0.000874,0.030438,0.018431,NaN
min,200.000000,0.026316,-0.236139,-0.108349,NaN
25%,273.500000,0.026316,-0.019915,-0.008416,NaN
50%,407.000000,0.026316,-0.006605,-0.003542,NaN
75%,667.000000,0.027027,-0.001590,-0.000735,NaN
max,11901.000000,0.033333,0.010636,0.064347,NaN


In [196]:
IPUMelast3 = pd.get_dummies(IPUMelast2, columns = ['year','statefip','ind'])
IPUMelast3['t'] = IPUMelast2['statefip']
IPUMelast3

,H_HL,H_HLTR,H_HLTR0,H_HLTR1,sex,age,citizen,empl,totemplNTR,totempl,...,ind_29.0,ind_30.0,ind_31.0,ind_32.0,ind_33.0,ind_34.0,ind_35.0,ind_36.0,ind_37.0,ind_38.0
660,-0.859249,-1.576648,-1.473306,-1.791759,0.695205,33.684932,0.013699,292,4190.0,4753.0,...,0,0,0,0,0,0,0,0,0,0
661,-0.962561,-1.572050,-1.445328,-1.796510,0.604046,32.806358,0.020231,346,4320.0,4997.0,...,0,0,0,0,0,0,0,0,0,0
662,-0.770628,-1.350955,-1.190986,-1.589635,0.530660,33.266509,0.009434,424,5334.0,6082.0,...,0,0,0,0,0,0,0,0,0,0
1944,-0.859249,-1.576648,-1.473306,-1.791759,0.695205,33.684932,0.013699,292,4190.0,4753.0,...,0,0,0,0,0,0,0,0,0,0
1945,-0.962561,-1.572050,-1.445328,-1.796510,0.604046,32.806358,0.020231,346,4320.0,4997.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14691,-1.250036,-2.397895,-2.740840,-2.319114,0.200000,43.843103,0.003448,580,NaN,3534.0,...,0,0,0,0,0,0,0,0,0,0
14692,-1.084891,-1.544055,-1.516348,-1.552087,0.258699,45.444781,0.007564,661,NaN,4842.0,...,0,0,0,0,0,0,0,0,0,0
20996,-1.143656,-2.225395,-1.864080,-2.339881,0.566489,40.364362,0.005319,376,2701.0,3443.0,...,0,0,0,0,0,0,0,1,0,0
20997,-1.250036,-2.397895,-2.740840,-2.319114,0.630769,43.174359,0.000000,390,2744.0,3534.0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
#local poly1 "_Iyear* _Istate* _Iind* "
#local poly "_Iyear* _Istate* ratio80 dage dcitizen dsex"
#local poly2 "_Iyear* _Istate*"

In [222]:
IPUMelast3['educH'] = IPUMelast3['dH_HL']
IPUMelast3['educHTR'] = IPUMelast3['dH_HLTR']
IPUMelast3['educHTR0'] = IPUMelast3['dH_HLTR0']

#*****IVs
IPUMelast3['educ_IVHLTR0'] = IPUMelast3['ddtotedu_IVHLTR0']
IPUMelast3['educ_IVHLTR'] = IPUMelast3['ddtotedu_IVHLTR']
IPUMelast3 = IPUMelast3.fillna(0)
IPUMelast3

,H_HL,H_HLTR,H_HLTR0,H_HLTR1,sex,age,citizen,empl,totemplNTR,totempl,...,ind_35.0,ind_36.0,ind_37.0,ind_38.0,t,educH,educHTR,educHTR0,educ_IVHLTR0,educ_IVHLTR
660,-0.859249,-1.576648,-1.473306,-1.791759,0.695205,33.684932,0.013699,292,4190.0,4753.0,...,0,0,0,0,8,0.000000,0.000000,0.000000,0.000000,0.000000
661,-0.962561,-1.572050,-1.445328,-1.796510,0.604046,32.806358,0.020231,346,4320.0,4997.0,...,0,0,0,0,8,-0.103312,0.004598,0.027978,0.094515,0.047391
662,-0.770628,-1.350955,-1.190986,-1.589635,0.530660,33.266509,0.009434,424,5334.0,6082.0,...,0,0,0,0,8,0.191933,0.221095,0.254342,0.238004,0.130675
1944,-0.859249,-1.576648,-1.473306,-1.791759,0.695205,33.684932,0.013699,292,4190.0,4753.0,...,0,0,0,0,8,0.000000,0.000000,0.000000,0.000000,0.000000
1945,-0.962561,-1.572050,-1.445328,-1.796510,0.604046,32.806358,0.020231,346,4320.0,4997.0,...,0,0,0,0,8,-0.103312,0.004598,0.027978,0.094515,0.047391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14691,-1.250036,-2.397895,-2.740840,-2.319114,0.200000,43.843103,0.003448,580,0.0,3534.0,...,0,0,0,0,932,-0.106380,-0.172500,-0.876760,0.093382,0.060024
14692,-1.084891,-1.544055,-1.516348,-1.552087,0.258699,45.444781,0.007564,661,0.0,4842.0,...,0,0,0,0,932,0.165146,0.853840,1.224492,0.248677,0.131744
20996,-1.143656,-2.225395,-1.864080,-2.339881,0.566489,40.364362,0.005319,376,2701.0,3443.0,...,0,1,0,0,932,0.000000,0.000000,0.000000,0.000000,0.000000
20997,-1.250036,-2.397895,-2.740840,-2.319114,0.630769,43.174359,0.000000,390,2744.0,3534.0,...,0,1,0,0,932,-0.106380,-0.172500,-0.876760,0.093382,0.060024


,year_1980,year_1990,year_2000,ind_1.0,ind_2.0,ind_3.0,ind_4.0,ind_5.0,ind_6.0,ind_8.0,...,ind_29.0,ind_30.0,ind_31.0,ind_32.0,ind_33.0,ind_34.0,ind_35.0,ind_36.0,ind_37.0,ind_38.0
660,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
661,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
662,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1944,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1945,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14691,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14692,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
20997,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [253]:
'''_Iyear = IPUMelast3.filter(regex='^year_',axis=1).head()
_Istate = IPUMelast3.filter(regex='^state_',axis=1).head()
_Iind = IPUMelast3.filter(regex='^ind_',axis=1).head()'''

In [262]:
'''_Iyear = _Iyear.reset_index().drop(columns='index')
_Istate = _Istate.reset_index().drop(columns='index')
_Iind = _Iind.reset_index().drop(columns='index')'''

In [278]:
'''poly1 = IPUMelast3.filter(regex=['^year_','^state_','^ind_'],axis=1).head()
poly1'''

"poly1 = IPUMelast3.filter(regex=['^year_','^state_','^ind_'],axis=1).head()\npoly1"

In [315]:
'''poly1 = datareg1[filter_col]
poly11 = poly1.loc[:, 'year_1980':'ind_38.0']
poly11'''

datareg1 = IPUMelast3[IPUMelast3['services'] == 2]
datareg1

filter_col = [col for col in datareg1 if col.startswith(('year_','statefip_','ind_'))]
poly1 = datareg1[filter_col]
cols = poly1.concat[datareg1["educH"]]
cols

AttributeError: 'DataFrame' object has no attribute 'concat'

In [311]:
#
#1st regression function and results
datareg1 = IPUMelast3[IPUMelast3['services'] == 2]
Y = datareg1["dshare_w"]
X = poly1
w = datareg1["empl80_"]
t = datareg1["t"]
X = sm.add_constant(X)
model = sm.WLS(Y,X,w,cluster=t)
results3 = model.fit()
print(results3.summary())

#datareg1["educH"],

                            WLS Regression Results                            
Dep. Variable:               dshare_w   R-squared:                       0.173
Model:                            WLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     2.845
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           7.33e-26
Time:                        12:44:52   Log-Likelihood:                 7457.0
No. Observations:                2295   AIC:                        -1.460e+04
Df Residuals:                    2137   BIC:                        -1.369e+04
Df Model:                         157                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.0006      0.000     -1.693   

In [ ]:

*if regress delta shares on H/L does not say on importance of consumption elasticities
reg dshare_w educH `poly1' [aw=empl80_] if services==2, r cluster(t)
reg dshare_w educH `poly1' [aw=empl80_] if services==1, r cluster(t) 
reg dshare_w educH `poly1' [aw=empl80_] if services==0, r cluster(t)
****